In [667]:
#install nfl_data_py package
!pip install nfl_data_py


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [668]:
import nfl_data_py as nfl


In [669]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tqdm import tqdm

import nfl_data_py as nfl

%matplotlib inline

In [670]:
from sklearn.metrics import mean_absolute_error

In [671]:
#Calculate the market-implied team total, derived the betting lines

def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [672]:
def last_name(full):
  if full == 'DanielThomas':
    return 'Thomas'
  elif full == 'JulioJones':
    return 'Jones'
  elif '.' not in full:
    print(full)
    return full
  else:
    return full.split(".",1)[1]

In [673]:
def team_fixer(team):
    if team == 'SD':
        return 'LAC'
    elif team == 'OAK':
        return 'LV'
    elif team == 'STL':
        return 'LA'
    else:
        return team

In [674]:
def snap_name_changer(player):
    if 'Jr.' in player:
        return player.split(' Jr.')[0]
        #return 'D.J. Davis'
    elif 'II' in player:
        return player.split(' II')[0]
    else:
        return player

In [675]:
def position_handler(pos):
  if pos in ['WR','RB','TE','FB']:
    return pos
  elif pos == 'RB/W':
    return 'RB'
  elif pos == 'WR/R':
    return 'WR'
  elif pos == 'HB':
    return 'RB'
  else:
    return 'other'

In [676]:
import joblib
import pickle


play_by_play_model = pickle.load(open('play_by_play.pkl', 'rb'))


In [677]:
in_season_model = pickle.load(open('model.pkl', 'rb'))

In [678]:
linear_model = pickle.load(open('linear_model.pkl', 'rb'))

# Apply Model

In [679]:
#load in data for NFL pass attempts

szn_receivers=[]
target_list=[]
  
df_list = []
weekly_list = []
target_week = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
week_list1= []


all_throws = []
n_szn = []
plus_1_szn = []

for YEAR in tqdm(range(2022,2023)):
    print(YEAR)
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    #data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','week','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch']]

    current_szn_df = df.dropna(subset=['receiver_player_name'])

  #Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 

    current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go']].set_index(['receiver_player_name','posteam'])
    current_szn_df['year'] = YEAR
  #apply model to the season's stats
    current_x = current_szn_df.drop(['yards_gained','year','week','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()
  
    
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    weekly_yac = yac_df.groupby(['receiver_player_name','posteam','week']).agg({'complete_pass':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})
    
    
    
    weekly_pass_offense=current_szn_df.groupby(['posteam','week']).agg({'week':'count','air_yards':'sum'})
    weekly_pass_offense.rename(columns={'week':'team_attempts','air_yards':'team_air_yards'},inplace=True)
    team_attempts=weekly_pass_offense.reset_index()[['posteam','week','team_attempts','team_air_yards']]
    team_attempts.set_index(['posteam','week'],inplace=True)
    weekly_rec_group = current_szn_df.groupby(['receiver_player_name','posteam','week']).agg({'week':'max','down':'count','complete_pass':'sum','yards_gained':'sum','air_yards':'sum','xYards':'sum','cp':'sum','implied_posteam_total':'max','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum'})
    weekly_rec_group.rename(columns={'week':'week#','down':'targets'},inplace=True)
    weekly_rec_group = weekly_rec_group.merge(weekly_yac[['yards_after_catch','xyac_mean_yardage']],right_index=True,left_index=True)
    
    
    feature_receivers = weekly_rec_group.reset_index().set_index(['posteam','week']).merge(team_attempts,right_index=True,left_index=True)
    feature_receivers = feature_receivers.reset_index().set_index(['receiver_player_name','posteam','week'])
    feature_receivers['szn'] = YEAR
    
    week_df1 = feature_receivers.copy()
    week_df = week_df1.copy()
    week_df = week_df.reset_index().set_index(['receiver_player_name','posteam'])
    feature_df = feature_receivers.copy()
    
    for target in tqdm(target_week):
        feature_receivers = feature_df[feature_df['week#']<target]
    
        player_list = []

        for x in feature_receivers.reset_index().set_index(['receiver_player_name','posteam']).index:
            player_list.append(x)
    
    
        players = list(set(player_list))

        print(target)
        for player in players:
            feature_receivers['xYards_median'] = feature_receivers['xYards']
            weekly_player = pd.DataFrame(feature_receivers.loc[player].sort_values('week#',ascending=False).head(6).agg({'yards_gained':'sum','targets':'sum','complete_pass':'sum','week#':'count','szn':'max','air_yards':'sum','xYards':'sum','cp':'sum','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum','team_attempts':'sum','team_air_yards':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum','xYards_median':'median'})).transpose()
            weekly_player['player'] = player[0]
            weekly_player['team'] = player[1]
            weekly_player['target_week'] = target
            weekly_player.set_index(['player','team','target_week','szn'],inplace=True)
            
          
            df_list.append(weekly_player)
        
        week_df = week_df1.copy()
        
        week_df = week_df[week_df['week#']==target]
        week_df = week_df.reset_index().rename(columns={'week':'target_week','receiver_player_name':'player','posteam':'team'})
        week_df = week_df.set_index(['player','team','target_week','szn'])

        week_list1.append(week_df)
        
        
        
        
        
weekly_receivers = pd.concat(df_list)
weekly_targets = pd.concat(week_list1)
        
        #weekly_receivers['aDOT']=weekly_receivers['air_yards']/weekly_receivers['targets']
        #weekly_receivers['AY_share']=round(weekly_receivers['air_yards']/weekly_receivers['team_air_yards'],3)
        #weekly_receivers['target_share']=round(weekly_receivers['targets']/weekly_receivers['team_attempts'],3)
        #weekly_receivers['yac/rec']=round(weekly_receivers['yards_after_catch']/weekly_receivers['complete_pass'],3)
        #weekly_receivers['xyac/rec']=round(weekly_receivers['xyac_mean_yardage']/weekly_receivers['complete_pass'],3)


        
        

  0%|                                                     | 0/1 [00:00<?, ?it/s]

2022



  0%|                                                    | 0/20 [00:00<?, ?it/s]

1
2



 10%|████▍                                       | 2/20 [00:04<00:39,  2.17s/it]

...


 90%|██████████████████████████████████████▋    | 18/20 [02:09<00:17,  8.50s/it]

19



 95%|████████████████████████████████████████▊  | 19/20 [02:18<00:08,  8.59s/it]

20



100%|████████████████████████████████████████████| 1/1 [02:56<00:00, 176.17s/it]


In [680]:
week_df1

,,,week#,targets,complete_pass,yards_gained,air_yards,xYards,cp,implied_posteam_total,goal_to_go,qb_hit,middle,yards_after_catch,xyac_mean_yardage,team_attempts,team_air_yards,szn
receiver_player_name,posteam,week,,,,,,,,,,,,,,,,
A.Green,ARI,1,1,4,2.0,13.0,42.0,35.595872,2.678442,24.0,0,0.0,1,0.0,8.527714,36,266.0,2022
A.Isabella,ARI,1,1,3,1.0,10.0,30.0,26.271294,2.055914,24.0,0,0.0,1,4.0,4.820307,36,266.0,2022
E.Benjamin,ARI,1,1,4,3.0,33.0,2.0,21.742391,3.079082,24.0,0,1.0,2,35.0,23.300142,36,266.0,2022
G.Dortch,ARI,1,1,9,7.0,63.0,62.0,65.930109,6.371063,24.0,0,1.0,0,31.0,40.471708,36,266.0,2022
J.Conner,ARI,1,1,6,5.0,29.0,7.0,36.797706,4.790721,24.0,0,1.0,1,38.0,41.558132,36,266.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C.Samuel,WAS,18,18,1,1.0,-2.0,-3.0,5.246031,0.875229,16.5,0,0.0,0,1.0,9.299595,18,158.0,2022
J.Dotson,WAS,18,18,4,3.0,72.0,60.0,32.024384,2.509778,16.5,0,0.0,1,49.0,10.645997,18,158.0,2022
J.Williams,WAS,18,18,2,2.0,9.0,3.0,10.941265,1.554412,16.5,0,0.0,1,6.0,12.949631,18,158.0,2022


In [681]:
feature_df['target_week'] = feature_df['week#']+1

feature_df.drop(('A.St. Brown','DET',7),inplace=True)

In [682]:
feature_df.loc['T.Lockett']

week#  targets  complete_pass  yards_gained  air_yards  \
posteam week                                                           
SEA     1         1        4            3.0          28.0       29.0   
        2         2       11            9.0         107.0      118.0   
        3         3       11            9.0          76.0      123.0   
        4         4        8            6.0          91.0      122.0   
        5         5        6            5.0         104.0      135.0   
        6         6        5            2.0          17.0       62.0   
        7         7        8            7.0          45.0       43.0   
        8         8        8            5.0          63.0       84.0   
        9         9        5            5.0          67.0       24.0   
        10       10        5            3.0          42.0       53.0   
        12       12        7            3.0          68.0       90.0   
        13       13       12            9.0         128.0       89.0   
        14       14        9            5.0          60.0       97.0   
        15       15        9            7.0          68.0       67.0   
        17       17        2            2.0          15.0        8.0   
        18       18        7            4.0          54.0       96.0   
        19       19        8            6.0          39.0       56.0   

                 xYards        cp  implied_posteam_total  goal_to_go  qb_hit  \
posteam week                                                                   
SEA     1     29.970425  2.799239                  19.00           0     0.0   
        2     80.492153  6.903809                  15.75           0     2.0   
        3     84.512757  6.664902                  22.00           0     1.0   
        4     65.110391  4.626942                  22.75           1     2.0   
        5     51.207771  3.103960                  20.00           0     2.0   
        6     39.778139  3.077915                  23.75           0     0.0   
        7     52.408489  5.625344                  23.00           0     1.0   
        8     51.552411  4.539161                  23.75           0     2.0   
        9     31.774134  3.406899                  24.00           1     1.0   
        10    37.522324  3.040777                  21.25           0     0.0   
        12    64.875953  4.536230                  25.75           0     1.0   
        13    89.451189  8.285791                  24.00           1     1.0   
        14    72.222910  5.002261                  24.00           1     1.0   
        15    66.285869  6.362094                  20.00           0     0.0   
        17    12.863331  1.584767                  21.00           0     0.0   
        18    61.932555  4.079475                  23.50           0     1.0   
        19    51.116833  5.581249                  16.00           1     0.0   

              middle  yards_after_catch  xyac_mean_yardage  team_attempts  \
posteam week                                                                
SEA     1          1                5.0          15.557277             27   
        2          3               22.0          36.641333             30   
        3          0                3.0          34.271528             44   
        4          2               11.0          22.844684             29   
        5          2                5.0          10.438336             24   
        6          2                8.0           7.939587             30   
        7          1                9.0          34.396273             26   
        8          0               28.0          27.092477             33   
        9          1               43.0          27.223590             32   
        10         1                9.0           8.453819             33   
        12         5               24.0           9.173747             37   
        13         2               68.0          43.275493             37   
        14         0                1.0       

In [683]:
    YEAR=2022
    
    
    snap_df = nfl.import_snap_counts([YEAR])
    snap_df['team'] = snap_df['team'].apply(lambda x: team_fixer(x))
    #snap_df = snap_df[snap_df['game_type']=='REG']
    snap_df = snap_df[['pfr_player_id','player','offense_snaps','offense_pct','week','team','game_id','position']]
    snap_df['player_copy'] = snap_df['player']
    snap_df.rename(columns={'player_copy':'full_name'},inplace=True)
    snap_df['full_name'] = snap_df['full_name'].apply(lambda x: snap_name_changer(x))
    
    
    snap_df['week#'] = snap_df['week']
    #snap_df = snap_df.set_index(['receiver_player_id','team','week'])
    snap_df = snap_df[snap_df['offense_snaps']>0]

    snap_df['last_name'] = snap_df['full_name'].apply(lambda x: x.split(" ",1)[1])
    snap_df['identifier'] = (snap_df['full_name']).str[0]+"."+snap_df['last_name']+"_"+snap_df['team']+"_"+(snap_df['week'].astype(str))
    snap_df = snap_df[['full_name','identifier','offense_snaps','position','team']]




In [684]:
#snap_df[snap_df['team']=='IND']['full_name'].value_counts()

In [685]:
snap_df = snap_df[snap_df['identifier']!='A.St. Brown_DET_7']

snap_df = snap_df[snap_df['identifier']!='D.Metcalf_SEA_7']

In [686]:
snap_df = snap_df[snap_df['identifier']!='M.Andrews_BAL_8']
snap_df = snap_df[snap_df['identifier']!='D.Bellinger_NYG_7']
snap_df = snap_df[snap_df['identifier']!='T.Higgins_CIN_14']
snap_df = snap_df[snap_df['identifier']!='D.Slayton_NYG_18']


In [687]:
snap_df[snap_df['identifier']=='D.Slayton_NYG_18']

,full_name,identifier,offense_snaps,position,team


In [688]:
from tqdm import tqdm
import tqdm.notebook as tq
import warnings
warnings.filterwarnings('ignore')

In [689]:
player_df_list = []



for player_id in tq.tqdm(feature_df.index):

  player = player_id[0:2]
  player_df = feature_df.loc[player]
  week_df=pd.DataFrame()
  week_df['week#']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
  player_df = player_df.merge(week_df,how='right')
  player_df['receiver_player_name'] = player[0]
  
  player_df['posteam'] = player[1]
  player_df['last'] = player_df['receiver_player_name'].apply(lambda x: last_name(x))
  player_df['identifier'] = (player_df['receiver_player_name']).str[0]+"."+player_df['last']+"_"+player_df['posteam']+"_"+player_df['week#'].astype(str)
  player_df = player_df.merge(snap_df,on='identifier')

  if player == 'A.St. Brown':
    player_df = player_df


  player_df.fillna(0,inplace=True)

  player_df['xYards_median'] = player_df['xYards'].rolling(6,min_periods=6).median()
  player_df['xYards/game'] = player_df['xYards'].rolling(6,min_periods=6).mean()

  player_df['xYards_sum'] = player_df['xYards'].rolling(6,min_periods=6).sum()
  player_df['yards_sum'] = player_df['yards_gained'].rolling(6,min_periods=6).sum()
  player_df['yards/game'] = player_df['yards_gained'].rolling(6,min_periods=6).mean()
  player_df['games'] = player_df['week#'].rolling(6,min_periods=1).count()
  player_df.rename(columns={'week#':'week'},inplace=True)
  target_df1 = pd.DataFrame()


  player_df['target_week'] = player_df['week']+1

  target_df1['target_week']=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

  player_df = player_df.merge(target_df1,how='right').sort_values('target_week')




  player_df = player_df.fillna(method='ffill')

  player_df_list.append(player_df)




  0%|          | 0/3945 [00:00<?, ?it/s]

In [690]:
trailing_weeks = pd.concat(player_df_list)


In [691]:
player_analyzer = trailing_weeks.drop_duplicates()

player_analyzer['aDOT'] = player_analyzer['air_yards']/player_analyzer['targets']
player_analyzer['target_share'] = player_analyzer['targets']/player_analyzer['team_attempts']




In [692]:
trailing_weeks = trailing_weeks.dropna()

trailing_weeks['games'].value_counts()

6.0    47712
Name: games, dtype: int64

In [693]:
trailing_weeks = trailing_weeks.drop_duplicates()

In [694]:
trailing_weeks['position'] = trailing_weeks['position'].apply(lambda x: position_handler(x))

In [695]:
one_hot = pd.get_dummies(trailing_weeks['position'])
one_hot.reset_index(inplace=True)
trailing_weeks = trailing_weeks.drop('position',axis = 1).reset_index()

trailing_weeks = trailing_weeks.join(one_hot,how='left', lsuffix='_left', rsuffix='_right')

#drop non WR/TE/RB/FB
trailing_weeks = trailing_weeks[trailing_weeks['other']==0]

In [696]:
trailing_weeks_all = trailing_weeks.copy()

In [697]:
trailing_weeks_all[['target_week']]

,target_week
0,9
1,10
2,11
3,12
4,13
...,...
4925,20
4926,17
4927,18
4928,19


In [698]:
np.mean([76.6,76.9,62.4,32,75.1,79.4])

67.06666666666666

## 5. Actual Results

### Week 8

In [699]:
trailing_weeks_all.rename(columns={'yards/game':'Yards/game'},inplace=True)

trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==8]

In [700]:
#trailing_weeks = weekly_receivers.reset_index()
#trailing_weeks = trailing_weeks[trailing_weeks['target_week']==8]



#trailing_weeks['xYards/game'] = trailing_weeks['xYards']/trailing_weeks['games']
#trailing_weeks['Yards/game'] = trailing_weeks['yards_gained']/trailing_weeks['games']
#trailing_weeks['target_share'] = trailing_weeks['targets']/trailing_weeks['team_attempts']
#trailing_weeks['AY_share'] = trailing_weeks['air_yards']/trailing_weeks['team_air_yards']
#trailing_weeks['yac/rec'] = trailing_weeks['yards_after_catch']/trailing_weeks['complete_pass']
#trailing_weeks['xyac/rec'] = trailing_weeks['xyac_mean_yardage']/trailing_weeks['complete_pass']


#trailing_weeks['aDOT']=trailing_weeks['air_yards']/trailing_weeks['targets']
#trailing_weeks['Yards/target'] = trailing_weeks['yards_gained']/trailing_weeks['targets']
#trailing_weeks['xYards/target'] = trailing_weeks['xYards']/trailing_weeks['targets']

#trailing_weeks.rename(columns={'AY_share':'AY_share_x','target_share':'target_share_x','yards_gained':'yards_gained_x','xYards':'xYards_x','Yards/target':'Yards/target_x','xYards/target':'xYards/target_x','team_attempts':'team_attempts_x','catch_rate':'catch_rate_x'},inplace=True)


#trailing_weeks['team_attempts/game_x'] = trailing_weeks['team_attempts_x']/trailing_weeks['games']


#trailing_weeks['targets/game_x'] = trailing_weeks['targets']/trailing_weeks['games']

#trailing_weeks['catch_rate_x'] = trailing_weeks['complete_pass']/trailing_weeks['targets']
#trailing_weeks['yac/rec_x'] = trailing_weeks['yards_after_catch']/trailing_weeks['complete_pass']


#trailing_weeks['cpoe_x'] = trailing_weeks['complete_pass']-trailing_weeks['cp']
#trailing_weeks['yacoe_x'] = trailing_weeks['yac/rec'] - trailing_weeks['xyac/rec']
#trailing_weeks['complete_pass_x'] = trailing_weeks['complete_pass']
#trailing_weeks['complete_pass_x'] = trailing_weeks['complete_pass']
#trailing_weeks['aDOT_x'] = trailing_weeks['aDOT']
#trailing_weeks['catches_per_game_x'] = trailing_weeks['complete_pass']/trailing_weeks['games']

In [701]:
trailing_weeks.dropna(inplace=True)

In [702]:
trailing_weeks

,index_left,week,targets,complete_pass,yards_gained,air_yards,xYards,cp,implied_posteam_total,goal_to_go,...,xYards_sum,yards_sum,Yards/game,games,index_right,FB,RB,TE,WR,other
13,6,7.0,5.0,4.0,21.0,-11.0,23.951324,4.043840,23.25,1.0,...,104.496487,113.0,18.833333,6.0,6,0,1,0,0,0
27,6,7.0,1.0,1.0,5.0,4.0,2.921464,0.659675,23.25,1.0,...,98.328598,146.0,24.333333,6.0,6,0,0,0,1,0
52,6,6.0,9.0,5.0,68.0,151.0,102.644805,5.008422,26.25,0.0,...,539.368438,485.0,80.833333,6.0,6,0,0,0,1,0
66,6,7.0,4.0,2.0,21.0,28.0,31.020628,2.912860,23.25,1.0,...,369.269097,306.0,51.000000,6.0,6,0,0,1,0,0
87,6,7.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,5.805800,2.0,0.333333,6.0,6,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4841,6,7.0,8.0,5.0,73.0,96.0,57.285837,4.606122,18.75,0.0,...,335.313037,382.0,63.666667,6.0,6,0,0,0,1,0
4855,6,7.0,1.0,1.0,4.0,1.0,3.224259,0.597253,18.75,0.0,...,22.639310,27.0,4.500000,6.0,6,0,0,0,1,0
4868,6,7.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,73.811671,73.0,12.166667,6.0,6,0,0,1,0,0
4882,6,6.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,82.495672,111.0,18.500000,6.0,6,0,0,0,1,0


In [703]:
features = ['xYards/game','Yards/game','RB','WR','FB']

linear_features = ['xYards/game','Yards/game']

In [704]:
prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['model_average']=(trailing_weeks['linear_projection']+trailing_weeks['gb_projection'])/2
trailing_weeks['pYards']=trailing_weeks['gb_projection']


In [705]:
projections = trailing_weeks.sort_values('pYards',ascending=False)

#[['player','team','pYards','Yards/game','games','target_share_x','aDOT','xYards/game','team_attempts_x']]

In [706]:
projections[projections['receiver_player_name']=='M.Andrews']

,index_left,week,targets,complete_pass,yards_gained,air_yards,xYards,cp,implied_posteam_total,goal_to_go,...,index_right,FB,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards
382,6,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6,0,0,1,0,0,66.843373,63.480619,65.161996,66.843373


In [707]:
df=pd.read_csv('betting_lines/nfl-best-bets.csv')
df = df[df['team']!='CLV']

mnf=pd.read_csv('betting_lines/nfl-best-bets (1).csv')

In [708]:
df = df.append(mnf)

In [709]:
projections.reset_index(inplace=True)

projections.rename(columns={'receiver_player_name':'player'},inplace=True)

In [710]:
names_to_change = ['M.Jones Jr','E.St','I.Smith Jr','A.St','T.Marshall Jr','A.Robinson II','M.Pittman Jr','K.Walker III','M.Gordon III']

def name_changer(name):
    if name in names_to_change:
        if name == 'M.Jones Jr':
            return 'M.Jones'
        if name == 'E.St':
            return 'E.St. Brown'
        if name == 'I.Smith Jr':
            return 'I.Smith'
        if name == 'A.St':
            return 'A.St. Brown'
        if name == 'T.Marshall Jr':
            return 'T.Marshall'
        if name == 'A.Robinson II':
            return 'Al.Robinson'
        if name == 'M.Pittman Jr':
            return 'M.Pittman'
        if name == 'K.Walker III':
            return 'K.Walker'
        if name == 'M.Gordon III':
            return 'M.Gordon'
    if name not in names_to_change:
        return name

In [711]:
odds = df[['player','team','line']]

def team_changer(name):
    if name == 'HST':
        return 'HOU'
    elif name == 'CLV':
        return 'CLE'
    elif name == 'ARZ':
        return 'ARI'
    elif name == 'BLT':
        return 'BAL'
    else:
        return name

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))



table = odds.merge(projections,how='left')

table = table.round(2)

table['diff'] = table['pYards'] - table['line']




table['abs_diff'] = abs(table['diff'])



In [712]:
table.sort_values('abs_diff',ascending=False).head(50)

,player,team,line,index,index_left,week,targets,complete_pass,yards_gained,air_yards,...,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards,diff,abs_diff
8,M.Jones,JAX,28.5,2275.0,6.0,7.0,8.0,4.0,57.0,122.0,...,0.0,0.0,1.0,0.0,48.49,47.62,48.05,48.49,19.99,19.99
80,C.Kupp,LA,92.5,2612.0,6.0,6.0,8.0,7.0,80.0,37.0,...,0.0,0.0,1.0,0.0,76.00,83.51,79.75,76.00,-16.50,16.50
58,M.Callaway,NO,39.5,3534.0,6.0,7.0,6.0,2.0,20.0,85.0,...,0.0,0.0,1.0,0.0,25.37,25.27,25.32,25.37,-14.13,14.13
89,M.Pittman,IND,57.5,2123.0,6.0,7.0,9.0,6.0,58.0,68.0,...,0.0,0.0,1.0,0.0,70.59,69.06,69.82,70.59,13.09,13.09
52,T.Conklin,NYJ,19.5,3831.0,6.0,7.0,6.0,4.0,22.0,45.0,...,0.0,1.0,0.0,0.0,31.47,32.23,31.85,31.47,11.97,11.97
40,D.Moore,CAR,60.5,649.0,6.0,7.0,10.0,7.0,69.0,88.0,...,0.0,0.0,1.0,0.0,48.62,47.66,48.14,48.62,-11.88,11.88
24,J.Jefferson,MIN,89.5,3197.0,6.0,6.0,8.0,6.0,107.0,41.0,...,0.0,0.0,1.0,0.0,77.64,80.55,79.09,77.64,-11.86,11.86
28,J.Reynolds,DET,39.5,1577.0,6.0,7.0,2.0,1.0,8.0,41.0,...,0.0,0.0,1.0,0.0,51.15,49.60,50.37,51.15,11.65,11.65
20,Z.Ertz,ARI,38.5,66.0,6.0,7.0,4.0,2.0,21.0,28.0,...,0.0,1.0,0.0,0.0,49.85,52.96,51.40,49.85,11.35,11.35
119,T.Higgins,CIN,76.5,1029.0,6.0,7.0,7.0,5.0,93.0,76.0,...,0.0,0.0,1.0,0.0,65.20,59.21,62.20,65.20,-11.30,11.30


In [713]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

  

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.95s/it]


In [714]:
#table.shape[0]


table = table.drop_duplicates()

In [715]:
table['pYards'].corr(table['line'])


0.9542687806165462

In [716]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==8]



100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.92s/it]


In [717]:
weekly_Rec = results.groupby('receiver_player_name').agg({'yards_gained':'sum','pass':'count','complete_pass':'sum'}).reset_index().rename(columns={'receiver_player_name':'player'})

In [718]:
table.drop(columns={'yards_gained'},inplace=True)

In [719]:
smash_table = table[table['abs_diff']>=10]

results = table.merge(weekly_Rec,on='player')


In [720]:
results=results.dropna()



In [721]:
results['Over_Picked'] =results['pYards']>results['line']

In [722]:
results['Over_Result'] = results['yards_gained'] > results['line']

results['Correct'] = results['Over_Picked'] == results['Over_Result']

In [723]:
#results = results[results['games']>1]



In [724]:
results['xYards/game'] = results['xYards_sum']/results['games']

In [725]:
w8_results = results[['player','team','games','line','pYards','xYards_median','xYards/game','yards_gained','Over_Picked','Over_Result','Correct','gb_projection','linear_projection']]

w8_results['week'] = 8

w8_results = w8_results[['player','team','games','line','pYards','xYards_median','xYards/game','yards_gained','Over_Picked','Over_Result','Correct','gb_projection','linear_projection']]


In [726]:
results['yards_gained'].corr(results['pYards'])

0.6184523789157267

In [727]:
#0.5860590594191387

In [728]:
import math
MSE = np.square(np.subtract(results['yards_gained'],results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (players w/ listed odds):\n")
print(RMSE)

print('-------------')

weekly_all  = weekly_Rec.merge(projections)[['player','pYards','yards_gained']]

MSE = np.square(np.subtract(weekly_all['yards_gained'],weekly_all['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (all players):\n")
print(RMSE)

Root Mean Square Error (players w/ listed odds):

30.8267751151495
-------------
Root Mean Square Error (all players):

21.9330332967768


In [729]:
#30.880608939504032
#-------------

#30.24493362852292

In [730]:
#overall
results['Correct'].value_counts()

True     53
False    47
Name: Correct, dtype: int64

In [731]:
smash_results = results[results['abs_diff']>=10]

smash_results['Correct'].value_counts()

False    8
True     7
Name: Correct, dtype: int64

In [732]:
smash_results[['player','line','xYards/game','pYards','abs_diff','yards_gained','Over_Picked','Over_Result','Correct']].sort_values('abs_diff',ascending=False)

,player,line,xYards/game,pYards,abs_diff,yards_gained,Over_Picked,Over_Result,Correct
8,M.Jones,28.5,55.080000,48.49,19.99,0.0,True,False,False
78,C.Kupp,92.5,86.716667,76.00,16.50,79.0,False,False,True
57,M.Callaway,39.5,28.781667,25.37,14.13,10.0,False,False,True
87,M.Pittman,57.5,73.588333,70.59,13.09,53.0,True,False,False
51,T.Conklin,19.5,31.301667,31.47,11.97,79.0,True,True,True
39,D.Moore,60.5,59.985000,48.62,11.88,152.0,False,True,False
23,J.Jefferson,89.5,76.630000,77.64,11.86,98.0,False,True,False
27,J.Reynolds,39.5,50.543333,51.15,11.65,14.0,True,False,False
19,Z.Ertz,38.5,61.545000,49.85,11.35,34.0,True,False,False
116,T.Higgins,76.5,59.608333,65.20,11.30,49.0,False,False,True


In [733]:
picked_overs = results[results['Over_Picked']==True]

picked_overs['Correct'].value_counts()

True     34
False    29
Name: Correct, dtype: int64

In [734]:
picked_unders = results[results['Over_Picked']==False]

picked_unders['Correct'].value_counts()

True     19
False    18
Name: Correct, dtype: int64

In [735]:
results['Over_Result'].value_counts()

True     52
False    48
Name: Over_Result, dtype: int64

In [736]:
smash_overs = smash_results[smash_results['Over_Picked']==True]


smash_overs['Correct'].value_counts()

False    6
True     3
Name: Correct, dtype: int64

In [737]:
smash_unders = smash_results[smash_results['Over_Picked']==False]


smash_unders['Correct'].value_counts()

True     4
False    2
Name: Correct, dtype: int64

In [738]:
table[table['team']=='CLE']

,player,team,line,index,index_left,week,targets,complete_pass,air_yards,xYards,...,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards,diff,abs_diff
121,A.Cooper,CLE,53.5,1098.0,6.0,7.0,4.0,3.0,84.0,35.33,...,0.0,0.0,1.0,0.0,62.39,62.37,62.38,62.39,8.89,8.89
122,D.Bell,CLE,15.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,D.Peoples-Jones,CLE,44.5,1140.0,6.0,7.0,6.0,6.0,59.0,41.68,...,0.0,0.0,1.0,0.0,41.06,39.96,40.51,41.06,-3.44,3.44
124,K.Hunt,CLE,12.5,1168.0,6.0,7.0,2.0,1.0,-6.0,11.01,...,1.0,0.0,0.0,0.0,13.64,13.88,13.76,13.64,1.14,1.14
125,N.Chubb,CLE,7.5,1182.0,6.0,7.0,3.0,2.0,-8.0,15.41,...,1.0,0.0,0.0,0.0,11.48,12.32,11.90,11.48,3.98,3.98


### Week 9

In [739]:
#trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==9]


trailing_weeks = trailing_weeks.dropna()

prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']



In [740]:
trailing_weeks.drop_duplicates().shape[0]

284

In [741]:
w9_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards_median','gb_projection','linear_projection']]

In [742]:
w9_table.rename(columns={'receiver_player_name':'player'},inplace=True)

In [743]:
w9_table.set_index(['player','team'],inplace=True)

In [744]:
week_by_week = feature_receivers.reset_index()

week_by_week.set_index(['receiver_player_name','posteam']).loc['A.St. Brown'][['week','targets','yards_gained','xYards','complete_pass','cp',]]

,week,targets,yards_gained,xYards,complete_pass,cp
posteam,,,,,,
DET,1,12,64.0,74.672700,8.0,8.883887
DET,2,12,116.0,76.962425,9.0,7.904535
DET,3,9,73.0,62.434424,6.0,6.727337
DET,5,6,18.0,31.988359,4.0,4.080900
DET,7,1,4.0,5.330446,1.0,0.883786
DET,8,10,69.0,75.051747,7.0,7.245048
DET,9,9,55.0,79.371461,4.0,6.638146
DET,10,11,119.0,76.603081,10.0,8.014863
DET,11,8,76.0,51.823136,7.0,5.685872


In [745]:
week_by_week = feature_receivers.reset_index()

sun_g = week_by_week.set_index(['receiver_player_name','posteam']).loc['A.St. Brown'][['week','targets','yards_gained','xYards','complete_pass','cp',]]

sun_g = sun_g[sun_g['targets']>1]

sun_g['xYards_trail'] = sun_g['xYards'].rolling(6,min_periods=6).mean()

sun_g

,week,targets,yards_gained,xYards,complete_pass,cp,xYards_trail
posteam,,,,,,,
DET,1,12,64.0,74.672700,8.0,8.883887,NaN
DET,2,12,116.0,76.962425,9.0,7.904535,NaN
DET,3,9,73.0,62.434424,6.0,6.727337,NaN
DET,5,6,18.0,31.988359,4.0,4.080900,NaN
DET,8,10,69.0,75.051747,7.0,7.245048,NaN
DET,9,9,55.0,79.371461,4.0,6.638146,66.746852
DET,10,11,119.0,76.603081,10.0,8.014863,67.068583
DET,11,8,76.0,51.823136,7.0,5.685872,62.878701
DET,12,10,122.0,80.189604,9.0,7.363505,65.837898


In [746]:
w9_odds = pd.read_csv('betting_lines/nfl-best-bets (3).csv').round(1)[['player','team','line']]
w9_odds = w9_odds[w9_odds['team']!='BLT']
w9_odds = w9_odds[w9_odds['team']!='NO']


w9_odds = w9_odds.append(pd.read_csv('betting_lines/nfl-best-bets (4).csv').round(1)[['player','team','line']])



odds = w9_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


In [747]:
odds.set_index(['player','team'],inplace=True)

In [748]:

w9_table = w9_table.merge(odds,right_index=True,left_index=True)
w9_table = w9_table.reset_index()

w9_table['abs. diff'] = abs(w9_table['pYards'] - w9_table['line'])

w9_table = w9_table.round(1)

w9_table['Over_Picked'] =w9_table['pYards']>w9_table ['line']

#w9_table.sort_values('abs. diff',ascending=False).head(50)


In [749]:
w9_table[w9_table['team']=='BAL']

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked
4,D.Duvernay,BAL,6.0,32.9,26.2,32.9,32.1,42.5,9.6,False
5,K.Drake,BAL,6.0,9.0,5.9,9.0,9.0,12.5,3.5,False
6,I.Likely,BAL,6.0,21.0,10.0,21.0,21.2,41.5,20.5,False
7,J.Oliver,BAL,6.0,8.6,3.3,8.6,9.0,7.5,1.1,True


In [750]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==9]


100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it]


In [751]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w9_table = w9_table.merge(results)

w9_table['Over_Result'] = w9_table['yards_gained']>w9_table['line']

w9_table['Win'] = (w9_table['Over_Result'] == w9_table['Over_Picked'])*1



In [752]:
w9_table = w9_table[w9_table['games']>1]

In [753]:

import math
MSE = np.square(np.subtract(w9_table['pYards'],w9_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w9_table['yards_gained'],w9_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

27.317402239530008
-----
Root Mean Square Error - Betting Line:

24.991510186368266


In [754]:
w9_table['week'] = 9

In [755]:
w9_table

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
0,Z.Ertz,ARI,6.0,46.1,51.0,46.1,48.2,39.5,6.6,True,40.0,True,1,9
1,D.London,ATL,6.0,37.1,46.2,37.1,34.8,38.5,1.4,False,23.0,False,1,9
2,K.Pitts,ATL,6.0,38.3,36.1,38.3,38.3,40.5,2.2,False,27.0,False,1,9
3,O.Zaccheaus,ATL,6.0,31.9,25.4,31.9,30.4,29.5,2.4,True,19.0,False,0,9
4,D.Duvernay,BAL,6.0,32.9,26.2,32.9,32.1,42.5,9.6,False,5.0,False,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,N.Westbrook-Ikhine,TEN,6.0,19.4,13.7,19.4,18.5,24.5,5.1,False,0.0,False,1,9
82,R.Woods,TEN,6.0,38.6,34.0,38.6,36.1,38.5,0.1,True,0.0,False,0,9
83,A.Gibson,WAS,6.0,20.8,20.1,20.8,22.1,22.5,1.7,False,11.0,False,1,9
84,C.Samuel,WAS,6.0,46.8,50.5,46.8,43.1,48.5,1.7,False,65.0,True,0,9


In [756]:
w9_table = w9_table[['player','team','games','week','line','pYards','xYards_median','Over_Picked','yards_gained','Over_Result','Win','gb_projection','linear_projection']]

In [757]:
w8_results['Correct'] = w8_results['Correct']*1
w8_results.rename(columns={'Correct':'Win'},inplace=True)

In [758]:
w8_results[w8_results['team']=='DET']

,player,team,games,line,pYards,xYards_median,xYards/game,yards_gained,Over_Picked,Over_Result,Win,gb_projection,linear_projection
27,J.Reynolds,DET,6.0,39.5,51.15,41.79,50.543333,14.0,True,False,0,51.15,49.60
28,K.Raymond,DET,6.0,29.5,27.54,25.69,27.521667,76.0,False,True,0,27.54,27.24
29,T.Hockenson,DET,6.0,42.5,45.51,44.75,48.906667,80.0,True,True,1,45.51,47.11


In [759]:
w8_results['week']=8

### Week 10

In [760]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==10]

trailing_weeks = trailing_weeks.dropna()


prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


In [761]:
trailing_weeks[trailing_weeks['team']=='ATL'].sort_values('pYards',ascending=False)[['receiver_player_name','games','xYards_median','pYards']].round(1)

,receiver_player_name,games,xYards_median,pYards
218,K.Pitts,6.0,49.3,46.4
191,D.London,6.0,40.9,34.2
232,O.Zaccheaus,6.0,25.4,30.2
306,D.Byrd,6.0,0.0,19.5
205,K.Hodge,6.0,0.0,12.4
271,T.Allgeier,6.0,2.7,9.6
258,B.Edwards,6.0,1.7,7.4
167,A.Williams,6.0,5.3,7.2
246,P.Hesse,6.0,3.6,6.7
318,C.Huntley,6.0,0.0,4.9


In [762]:
trailing_weeks[trailing_weeks['team']=='CAR'].sort_values('pYards',ascending=False)[['receiver_player_name','games','xYards_median','pYards']].round(1)

,receiver_player_name,games,xYards_median,pYards
651,D.Moore,6.0,69.9,60.9
679,R.Anderson,6.0,40.7,38.0
637,C.McCaffrey,6.0,31.1,34.6
754,T.Marshall,6.0,25.9,28.9
707,T.Tremble,6.0,17.4,19.1
693,S.Smith,6.0,0.0,13.1
665,I.Thomas,6.0,6.9,10.0
733,S.Sullivan,6.0,0.0,6.8
721,G.Ricci,6.0,0.0,6.5
782,D.Foreman,6.0,0.0,6.5


In [763]:
player = 'V.Jefferson'

week_by_week['aDOT'] = week_by_week['air_yards']/week_by_week['targets']
all_szn = week_by_week[week_by_week['receiver_player_name']==player].sort_values('week',ascending=False)[['week','targets','aDOT','xYards','yards_gained']].round(1)

all_szn

trailing_6 = all_szn.head(6)

t_mean = trailing_6['yards_gained'].mean()
t_mean_x = trailing_6['xYards'].mean()

t_median = trailing_6['yards_gained'].median()
t_median_x = trailing_6['xYards'].median()

mean = all_szn['yards_gained'].mean()
mean_x = all_szn['xYards'].mean()

median = all_szn['yards_gained'].median()
median_x = all_szn['xYards'].median()

print('{}'.format(player))

display(all_szn.set_index('week'))


print('xYards Per Game (Trailing 6 Games): {}'.format(round(t_mean_x,1)))
print('Median xYards (Trailing 6 Games): {}\n'.format(round(t_median_x,1)))

print('Yards Per Game (Trailing 6 Games): {}'.format(round(t_mean,1)))
print('Median Yards (Trailing 6 Games): {}'.format(round(t_median,1)))



V.Jefferson


,targets,aDOT,xYards,yards_gained
week,,,,
18,7,19.1,69.2,61.0
17,3,25.3,35.5,77.0
16,5,6.8,30.5,19.0
15,2,16.0,21.1,32.0
14,4,17.2,38.5,44.0
13,4,17.8,47.2,39.0
12,6,9.5,38.0,29.0
11,5,13.0,43.8,41.0
10,3,7.7,21.1,27.0


xYards Per Game (Trailing 6 Games): 40.3
Median xYards (Trailing 6 Games): 37.0

Yards Per Game (Trailing 6 Games): 45.3
Median Yards (Trailing 6 Games): 41.5


In [764]:
w10_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards_median','gb_projection','linear_projection']]


w10_table.rename(columns={'receiver_player_name':'player'},inplace=True)

In [765]:
w10_table.set_index(['player','team'],inplace=True)

In [766]:
w10_odds = pd.read_csv('betting_lines/nfl-best-bets (5).csv').round(1)[['player','team','line']]






odds = w10_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w10_table = w10_table[w10_table['games']>1]
w10_table = w10_table.merge(odds,right_index=True,left_index=True)
w10_table = w10_table.reset_index()

w10_table['abs. diff'] = abs(w10_table['pYards'] - w10_table['line'])

w10_table = w10_table.round(1)

w10_table['Over_Picked'] =w10_table['pYards']>w10_table ['line']

w10_table.sort_values('abs. diff',ascending=True).tail(25)

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked
20,M.Gordon,DEN,6.0,18.3,19.3,18.3,18.6,11.5,6.8,True
84,D.Samuel,SF,6.0,57.3,53.1,57.3,53.2,50.5,6.8,True
17,P.Hendershot,DAL,6.0,14.6,9.4,14.6,15.7,7.5,7.1,True
3,I.McKenzie,BUF,6.0,29.0,30.3,29.0,28.7,21.5,7.5,True
47,D.Adams,LV,6.0,74.8,74.4,74.8,79.1,82.5,7.7,False
16,N.Brown,DAL,6.0,39.4,44.3,39.4,38.8,31.5,7.9,True
54,T.Hill,MIA,6.0,102.5,123.8,102.5,106.9,94.5,8.0,True
87,K.Juszczyk,SF,6.0,17.6,11.4,17.6,19.6,9.5,8.1,True
55,T.Sherfield,MIA,6.0,27.7,25.6,27.7,26.9,19.5,8.2,True
27,R.Tonyan,GB,6.0,38.1,33.2,38.1,36.7,29.5,8.6,True


In [767]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==10]


100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.29s/it]


In [768]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w10_table = w10_table.merge(results)

w10_table['Over_Result'] = w10_table['yards_gained']>w10_table['line']

w10_table['Win'] = (w10_table['Over_Result'] == w10_table['Over_Picked'])*1

w10_table = w10_table[w10_table['games']>1]

w10_table['week'] = 10


In [769]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.96632178845888
-----
Root Mean Square Error - Betting Line:

29.502524233872137


In [770]:
w10_table[w10_table['team']=='IND']

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
27,J.Taylor,IND,6.0,18.1,22.3,18.1,18.7,16.5,1.6,True,16.0,False,0,10
28,M.Pittman,IND,6.0,54.2,60.8,54.2,55.8,53.5,0.7,True,53.0,False,0,10
29,P.Campbell,IND,6.0,40.9,33.5,40.9,38.1,25.5,15.4,True,76.0,True,1,10
30,A.Pierce,IND,6.0,48.9,51.5,48.9,46.6,27.5,21.4,True,0.0,False,0,10


### Week 11

In [771]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==11]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

In [772]:
w11_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]

w11_table.rename(columns={'receiver_player_name':'player'},inplace=True)

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (8).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table[w11_table['games']>1]
w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


def o_u(over):
  if over == True:
    return 'Over'
  elif over == False:
    return 'Under'



In [773]:
tnf = w11_table[(w11_table['team']=='GB')|(w11_table['team']=='TEN')]


In [774]:
w11_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]

w11_table.rename(columns={'receiver_player_name':'player'},inplace=True)

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (9).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table[w11_table['games']>1]
w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


w11_table = w11_table.append(tnf)



In [775]:
snf = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]

snf.rename(columns={'receiver_player_name':'player'},inplace=True)

snf.set_index(['player','team'],inplace=True)

snf_odds = pd.read_csv('betting_lines/nfl-best-bets (10).csv').round(1)[['player','team','line']]






odds = snf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

snf = snf[snf['games']>1]
snf = snf.merge(odds,right_index=True,left_index=True)
snf = snf.reset_index()

snf['abs. diff'] = abs(snf['pYards'] - snf['line'])

snf = snf.round(1)

snf['Over_Picked'] =snf['pYards']>snf ['line']


primetime = snf[(snf['team']=='KC')|(snf['team']=='LAC')]

In [776]:
w11_table = w11_table.append(primetime)

In [777]:
mnf = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]

mnf.rename(columns={'receiver_player_name':'player'},inplace=True)

mnf.set_index(['player','team'],inplace=True)

mnf_odds = pd.read_csv('betting_lines/nfl-best-bets (11).csv').round(1)[['player','team','line']]






odds = mnf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

mnf = mnf[mnf['games']>1]
mnf = mnf.merge(odds,right_index=True,left_index=True)
mnf = mnf.reset_index()

mnf['abs. diff'] = abs(mnf['pYards'] - mnf['line'])

mnf = mnf.round(1)
mnf['Over_Picked'] =mnf['pYards']>mnf ['line']


mnf = mnf[(mnf['team']=='ARI')|(mnf['team']=='SF')]

In [778]:
mnf

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,J.Conner,ARI,6.0,19.9,19.4,0,20.5,18.3,20.1,19.9,19.5,0.4,True
1,R.Moore,ARI,6.0,61.1,56.6,1,60.0,67.2,56.2,61.1,59.5,1.6,True
2,T.McBride,ARI,6.0,4.9,0.9,0,0.0,1.2,4.6,4.9,19.5,14.6,False
3,B.Aiyuk,SF,6.0,64.0,56.4,1,55.1,70.8,57.4,64.0,54.5,9.5,True
4,D.Samuel,SF,6.0,52.4,56.4,1,55.1,58.8,53.1,52.4,52.5,0.1,False
5,J.Jennings,SF,6.0,25.9,23.6,1,23.4,25.7,24.8,25.9,14.5,11.4,True
6,G.Kittle,SF,6.0,39.9,42.1,0,36.2,52.0,43.5,39.9,41.5,1.6,False
7,K.Juszczyk,SF,6.0,17.6,13.4,0,11.4,25.5,19.6,17.6,7.5,10.1,True


In [779]:
w11_table = w11_table.append(mnf)

In [780]:
w11_table[(w11_table['team']=='ARI')|(w11_table['team']=='SF')]

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
0,J.Conner,ARI,6.0,19.9,19.4,0,20.5,18.3,19.9,20.1,19.5,0.4,True
1,R.Moore,ARI,6.0,61.1,56.6,1,60.0,67.2,61.1,56.2,59.5,1.6,True
2,T.McBride,ARI,6.0,4.9,0.9,0,0.0,1.2,4.9,4.6,19.5,14.6,False
3,B.Aiyuk,SF,6.0,64.0,56.4,1,55.1,70.8,64.0,57.4,54.5,9.5,True
4,D.Samuel,SF,6.0,52.4,56.4,1,55.1,58.8,52.4,53.1,52.5,0.1,False
5,J.Jennings,SF,6.0,25.9,23.6,1,23.4,25.7,25.9,24.8,14.5,11.4,True
6,G.Kittle,SF,6.0,39.9,42.1,0,36.2,52.0,39.9,43.5,41.5,1.6,False
7,K.Juszczyk,SF,6.0,17.6,13.4,0,11.4,25.5,17.6,19.6,7.5,10.1,True


In [781]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
19,R.Cobb,GB,6.0,39.4,37.5,1,28.2,42.8,39.4,37.9,21.5,17.9,True
18,C.Watson,GB,6.0,25.7,23.6,1,16.4,26.8,25.7,25.2,43.5,17.8,False
15,D.Peoples-Jones,CLE,6.0,64.0,57.8,1,51.5,74.3,64.0,59.4,46.5,17.5,True
13,T.Higgins,CIN,6.0,57.0,52.5,1,59.5,62.2,57.0,52.3,74.5,17.5,False
21,S.Watkins,GB,6.0,30.7,28.7,1,26.9,34.3,30.7,30.4,14.5,16.2,True
38,J.Meyers,NE,6.0,64.4,59.4,1,58.2,67.0,64.4,57.6,48.5,15.9,True
22,A.St. Brown,DET,6.0,65.0,67.1,1,75.8,75.0,65.0,64.3,80.5,15.5,False
2,T.McBride,ARI,6.0,4.9,0.9,0,0.0,1.2,4.9,4.6,19.5,14.6,False
22,A.Lazard,GB,6.0,63.8,67.1,1,76.1,69.0,63.8,62.1,49.5,14.3,True
18,D.Schultz,DAL,6.0,27.5,27.3,0,27.1,32.5,27.5,29.1,41.5,14.0,False


In [782]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==11]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w11_table = w11_table.merge(results)

w11_table['Over_Result'] = w11_table['yards_gained']>w11_table['line']

w11_table['Win'] = (w11_table['Over_Result'] == w11_table['Over_Picked'])*1

w11_table = w11_table[w11_table['games']>1]

w11_table['week'] = 11

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.70s/it]


In [783]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
57,R.Cobb,GB,6.0,39.4,37.5,1,28.2,42.8,39.4,37.9,21.5,17.9,True,73.0,True,1,11
56,C.Watson,GB,6.0,25.7,23.6,1,16.4,26.8,25.7,25.2,43.5,17.8,False,48.0,True,0,11
15,D.Peoples-Jones,CLE,6.0,64.0,57.8,1,51.5,74.3,64.0,59.4,46.5,17.5,True,61.0,True,1,11
13,T.Higgins,CIN,6.0,57.0,52.5,1,59.5,62.2,57.0,52.3,74.5,17.5,False,148.0,True,0,11
59,S.Watkins,GB,6.0,30.7,28.7,1,26.9,34.3,30.7,30.4,14.5,16.2,True,0.0,False,0,11
38,J.Meyers,NE,6.0,64.4,59.4,1,58.2,67.0,64.4,57.6,48.5,15.9,True,52.0,True,1,11
22,A.St. Brown,DET,6.0,65.0,67.1,1,75.8,75.0,65.0,64.3,80.5,15.5,False,76.0,False,1,11
74,T.McBride,ARI,6.0,4.9,0.9,0,0.0,1.2,4.9,4.6,19.5,14.6,False,14.0,False,1,11
60,A.Lazard,GB,6.0,63.8,67.1,1,76.1,69.0,63.8,62.1,49.5,14.3,True,57.0,True,1,11
41,C.Olave,NO,6.0,64.5,67.0,1,59.0,65.0,64.5,60.7,50.5,14.0,True,102.0,True,1,11


In [784]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.96632178845888
-----
Root Mean Square Error - Betting Line:

29.502524233872137


### Week 12

In [785]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==12]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

w12_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w12_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (12).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table[w12_table['games']>1]
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 



In [786]:
tg = w12_table[w12_table['team']!='CAR'].copy()

In [787]:
prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

w12_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w12_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (13).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table[w12_table['games']>1]
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']

w12_table = w12_table.append(tg)


In [788]:


mnf12 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
mnf12.rename(columns={'receiver_player_name':'player'},inplace=True)
 
mnf12.set_index(['player','team'],inplace=True)
 
mnf12_odds = pd.read_csv('betting_lines/nfl-best-bets (14).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf12 = mnf12.merge(odds,right_index=True,left_index=True)
mnf12 = mnf12.reset_index()
 
mnf12['abs. diff'] = abs(mnf12['pYards'] - mnf12['line'])
 
mnf12 = mnf12.round(1)
 
mnf12['Over_Picked'] =mnf12['pYards']>mnf12 ['line']

w12_table = w12_table.append(mnf12)


In [789]:
w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])
w12_table[(w12_table['abs. median diff']>=10)&(w12_table['abs. diff']>=10)]

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked,abs. median diff
11,A.Cooper,CLE,6.0,65.5,65.4,1,75.7,78.3,65.5,64.6,51.5,14.0,True,24.2
19,A.Lazard,GB,6.0,61.8,70.3,1,76.1,59.2,61.8,60.3,46.5,15.3,True,29.6
44,T.Hill,MIA,6.0,75.4,100.6,1,104.3,111.8,75.4,94.3,92.5,17.1,False,11.8
49,G.Wilson,NYJ,6.0,38.2,36.3,1,29.3,46.3,38.2,38.6,53.5,15.3,False,24.2
51,A.Brown,PHI,6.0,55.9,59.3,1,51.5,63.5,55.9,56.3,71.0,15.1,False,19.5
59,T.Lockett,SEA,6.0,45.8,44.0,1,45.5,56.3,45.8,46.0,63.5,17.7,False,18.0
67,M.Evans,TB,6.0,73.5,82.3,1,77.2,72.7,73.5,71.1,60.5,13.0,True,16.7
2,G.Davis,BUF,6.0,71.4,70.9,1,71.8,79.0,71.4,67.7,58.5,12.9,True,13.3


In [790]:
mnf12

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,J.Taylor,IND,6.0,17.9,18.9,0,19.0,12.3,17.7,17.9,13.5,4.4,True
1,M.Alie-Cox,IND,6.0,6.1,3.1,0,0.0,4.5,6.9,6.1,13.5,7.4,False
2,M.Pittman,IND,6.0,63.6,65.2,1,61.1,65.8,60.1,63.6,67.5,3.9,False
3,P.Campbell,IND,6.0,50.2,47.0,1,50.9,54.7,46.9,50.2,48.5,1.7,True
4,A.Pierce,IND,6.0,34.8,36.2,1,42.0,33.7,34.0,34.8,35.5,0.7,False
5,D.Johnson,PIT,6.0,49.1,52.6,1,46.0,41.5,45.1,49.1,45.5,3.6,True
6,G.Pickens,PIT,6.0,39.2,36.1,1,37.3,47.7,38.9,39.2,43.5,4.3,False
7,N.Harris,PIT,6.0,18.7,19.2,0,19.6,15.0,18.8,18.7,17.5,1.2,True
8,P.Freiermuth,PIT,6.0,48.6,55.7,0,58.4,57.3,52.2,48.6,48.5,0.1,True
9,Z.Gentry,PIT,6.0,12.9,12.3,0,7.5,9.7,13.4,12.9,7.5,5.4,True


In [791]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==12]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w12_table = w12_table.merge(results)

w12_table['Over_Result'] = w12_table['yards_gained']>w12_table['line']

w12_table['Win'] = (w12_table['Over_Result'] == w12_table['Over_Picked'])*1

w12_table = w12_table[w12_table['games']>1]

w12_table['week'] = 12

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.88s/it]


### Week 13

In [792]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==13]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


w13_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w13_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (16).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['pYards'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['pYards']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf = w13_table.copy()

In [793]:
w13_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
w13_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (20).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['gb_projection'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['gb_projection']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf.append(w13_table[(w13_table['team']!='BUF')&(w13_table['team']!='NE')])

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
0,D.Knox,BUF,6.0,34.4,33.0,0,27.6,36.0,34.4,33.2,32.5,1.9,True
1,D.Singletary,BUF,6.0,17.4,16.0,0,12.9,13.5,17.4,16.6,11.5,5.9,True
2,G.Davis,BUF,6.0,53.5,66.8,1,67.6,56.8,53.5,57.7,48.5,5.0,True
3,I.McKenzie,BUF,6.0,28.2,28.8,1,21.9,27.0,28.2,27.9,32.5,4.3,False
4,S.Diggs,BUF,6.0,76.4,97.1,1,108.8,100.3,76.4,88.5,81.5,5.1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,D.Henry,TEN,6.0,16.2,11.4,0,13.2,26.2,16.2,18.8,14.5,1.7,True
115,C.Samuel,WAS,6.0,30.1,26.5,1,31.7,34.3,30.1,29.3,30.5,0.4,False
116,J.Dotson,WAS,6.0,24.4,25.7,1,25.7,23.2,24.4,25.0,19.5,4.9,True
117,L.Thomas,WAS,6.0,19.1,17.2,0,17.9,17.2,19.1,18.5,23.5,4.4,False


In [794]:
w13_table = w13_table[w13_table['team']!='IND']

In [795]:
w13_table = w13_table[w13_table['team']!='DAL']

w13_table = w13_table[w13_table['team']!='NO']

w13_table = w13_table[w13_table['team']!='TB']

In [796]:
w13 = trailing_weeks.sort_values('pYards',ascending=False)[['full_name','team','pYards','xYards_median','xYards/game','Yards/game','linear_projection']].round(1)

#w13[(w13['team']=='IND')|(w13['team']=='DAL')]

In [797]:
snf13 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
snf13.rename(columns={'receiver_player_name':'player'},inplace=True)
 
snf13.set_index(['player','team'],inplace=True)
 
snf13_odds = pd.read_csv('betting_lines/nfl-best-bets (21).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf13 = snf13.merge(odds,right_index=True,left_index=True)
snf13 = snf13.reset_index()
 
snf13['abs. diff'] = abs(snf13['gb_projection'] - snf13['line'])
 
snf13 = snf13.round(1)
 
snf13['Over_Picked'] =snf13['gb_projection']>snf13 ['line']

In [798]:
mnf13 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
mnf13.rename(columns={'receiver_player_name':'player'},inplace=True)
 
mnf13.set_index(['player','team'],inplace=True)
 
mnf13_odds = pd.read_csv('betting_lines/nfl-best-bets (23).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf13 = mnf13.merge(odds,right_index=True,left_index=True)
mnf13 = mnf13.reset_index()
 
mnf13['abs. diff'] = abs(mnf13['gb_projection'] - mnf13['line'])
 
mnf13 = mnf13.round(1)
 
mnf13['Over_Picked'] = mnf13['gb_projection']>mnf13 ['line']

In [799]:
#mnf13

In [800]:
w13_table = w13_table.append(snf13)

w13_table = w13_table.append(mnf13)

In [801]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==13]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w13_table = w13_table.merge(results)

w13_table['Over_Result'] = w13_table['yards_gained']>w13_table['line']

w13_table['Win'] = (w13_table['Over_Result'] == w13_table['Over_Picked'])*1

w13_table = w13_table[w13_table['games']>1]

w13_table['week'] = 13

100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.01s/it]


In [802]:
w13_table[w13_table['abs. diff']>=10].sort_values('abs. diff',ascending=False).head(50)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
51,D.Adams,LV,6.0,71.2,100.6,1,95.0,97.5,89.2,71.2,95.5,24.3,False,177.0,True,0,13
66,G.Wilson,NYJ,6.0,44.2,41.0,1,42.8,57.7,45.0,44.2,62.5,18.3,False,162.0,True,0,13
82,T.Lockett,SEA,6.0,48.1,46.3,1,45.7,50.3,45.0,48.1,63.5,15.4,False,128.0,True,0,13
113,J.Landry,NO,6.0,27.3,30.7,1,27.5,24.0,27.8,27.3,40.5,13.2,False,14.0,False,1,13
69,A.Brown,PHI,6.0,61.4,57.7,1,48.8,65.8,56.3,61.4,74.5,13.1,False,119.0,True,0,13
46,T.Kelce,KC,6.0,72.3,75.3,0,63.2,94.2,75.2,72.3,84.5,12.2,False,56.0,False,1,13
20,C.Sutton,DEN,6.0,48.7,51.2,1,53.9,45.2,45.7,48.7,60.5,11.8,False,0.0,False,1,13
12,S.Perine,CIN,6.0,19.7,17.5,0,14.1,19.8,19.6,19.7,31.5,11.8,False,49.0,True,0,13
5,D.Robinson,BAL,6.0,41.2,40.9,1,40.9,41.3,39.1,41.2,29.5,11.7,True,41.0,True,1,13
120,C.Otton,TB,6.0,28.2,26.2,0,29.6,34.2,29.1,28.2,16.5,11.7,True,28.0,True,1,13


### Week 14

In [803]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==14]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


w14_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (25).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,right_index=True,left_index=True)
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['gb_projection'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['gb_projection']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
tnf = w14_table[(w14_table['team']=='LA')|(w14_table['team']=='LV')]




In [804]:


w14_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (27).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,right_index=True,left_index=True)
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['gb_projection'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['gb_projection']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
w14_table = tnf.append(w14_table)

w14_table = w14_table[(w14_table['team']!='LAC') & (w14_table['team']!='MIA')]

w14_table = w14_table[(w14_table['team']!='ARI') & (w14_table['team']!='NE')]

In [805]:
pd.read_csv('betting_lines/nfl-best-bets (28).csv').round(1)[['player','team','line']]


,player,team,line
0,A. Ekeler,LAC,42.5
1,G. Everett,LAC,33.5
2,J. Palmer,LAC,47.5
3,K. Allen,LAC,64.5
4,M. Williams,LAC,47.5
5,J. Waddle,MIA,70.5
6,J. Wilson Jr.,MIA,11.5
7,M. Gesicki,MIA,19.5
8,R. Mostert,MIA,8.5
9,T. Sherfield,MIA,32.5


In [806]:


snf14 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
snf14.rename(columns={'receiver_player_name':'player'},inplace=True)
 
snf14.set_index(['player','team'],inplace=True)
 
snf14_odds = pd.read_csv('betting_lines/nfl-best-bets (28).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf14 = snf14.merge(odds,right_index=True,left_index=True)
snf14 = snf14.reset_index()
 
snf14['abs. diff'] = abs(snf14['gb_projection'] - snf14['line'])
 
snf14 = snf14.round(1)
 
snf14['Over_Picked'] =snf14['gb_projection']>snf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
snf14 = snf14[(snf14['team']=='LAC')|(snf14['team']=='MIA')]

In [807]:
w14_table = w14_table.append(snf14)

In [808]:


mnf14 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
mnf14.rename(columns={'receiver_player_name':'player'},inplace=True)
 
mnf14.set_index(['player','team'],inplace=True)
 
mnf14_odds = pd.read_csv('betting_lines/nfl-best-bets (30).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf14 = mnf14.merge(odds,right_index=True,left_index=True)
mnf14 = mnf14.reset_index()
 
mnf14['abs. diff'] = abs(mnf14['gb_projection'] - mnf14['line'])
 
mnf14 = mnf14.round(1)
 
mnf14['Over_Picked'] =mnf14['gb_projection']>mnf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
mnf14 = mnf14[(mnf14['team']=='ARI')|(mnf14['team']=='NE')]

In [809]:
w14_table = w14_table.append(mnf14)

In [810]:
w14_table.sort_values('abs. diff',ascending=False).head(50)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
6,J.Palmer,LAC,6.0,70.7,73.4,1,73.6,71.5,66.2,70.7,47.5,23.2,True
31,D.Slayton,NYG,6.0,67.0,59.7,1,57.7,76.3,61.0,67.0,45.5,21.5,True
14,T.Hill,MIA,6.0,80.0,107.9,1,108.5,113.0,98.4,80.0,100.5,20.5,False
45,J.Jefferson,MIN,6.0,74.4,84.7,1,85.0,103.8,83.4,74.4,94.5,20.1,False
62,M.Evans,TB,6.0,70.6,84.1,1,82.4,67.2,70.1,70.6,51.5,19.1,True
6,N.Agholor,NE,6.0,19.8,18.0,1,13.5,17.0,18.9,19.8,33.5,13.7,False
76,R.Woods,TEN,6.0,25.9,30.1,1,38.8,21.2,26.5,25.9,39.5,13.6,False
15,C.Otton,TB,6.0,31.7,31.9,0,34.3,35.0,32.3,31.7,18.5,13.2,True
5,M.Brown,ARI,6.0,69.6,90.6,1,88.5,81.3,78.4,69.6,56.5,13.1,True
10,M.Williams,LAC,6.0,60.1,62.5,1,66.6,64.5,58.2,60.1,47.5,12.6,True


In [811]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==14]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w14_table = w14_table.merge(results)

w14_table['Over_Result'] = w14_table['yards_gained']>w14_table['line']

w14_table['Win'] = (w14_table['Over_Result'] == w14_table['Over_Picked'])*1

w14_table = w14_table[w14_table['games']>1]

w14_table['week'] = 14

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it]


In [812]:
w14_table.sort_values('abs. diff',ascending=False).head(30)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
86,J.Palmer,LAC,6.0,70.7,73.4,1,73.6,71.5,66.2,70.7,47.5,23.2,True,53.0,True,1,14
36,D.Slayton,NYG,6.0,67.0,59.7,1,57.7,76.3,61.0,67.0,45.5,21.5,True,42.0,False,0,14
91,T.Hill,MIA,6.0,80.0,107.9,1,108.5,113.0,98.4,80.0,100.5,20.5,False,81.0,False,1,14
48,J.Jefferson,MIN,6.0,74.4,84.7,1,85.0,103.8,83.4,74.4,94.5,20.1,False,233.0,True,0,14
62,M.Evans,TB,6.0,70.6,84.1,1,82.4,67.2,70.1,70.6,51.5,19.1,True,44.0,False,0,14
99,N.Agholor,NE,6.0,19.8,18.0,1,13.5,17.0,18.9,19.8,33.5,13.7,False,32.0,False,1,14
74,R.Woods,TEN,6.0,25.9,30.1,1,38.8,21.2,26.5,25.9,39.5,13.6,False,49.0,True,0,14
22,C.Otton,TB,6.0,31.7,31.9,0,34.3,35.0,32.3,31.7,18.5,13.2,True,28.0,True,1,14
98,M.Brown,ARI,6.0,69.6,90.6,1,88.5,81.3,78.4,69.6,56.5,13.1,True,34.0,False,0,14
89,M.Williams,LAC,6.0,60.1,62.5,1,66.6,64.5,58.2,60.1,47.5,12.6,True,116.0,True,1,14


### Week 15

In [813]:
beta_table = pd.read_csv('wk15_beta.csv')


In [814]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==15]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (33).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 




In [815]:
beta_table.shape[0]

387

In [816]:
weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])

In [817]:
weekly_table['linear_diff']=abs(weekly_table['linear_projection']-weekly_table['line'])
weekly_table['beta_diff']=abs(weekly_table['beta_yards']-weekly_table['line'])
weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)
weekly_table['consensus_diff']=abs(weekly_table['consensus_proj']-weekly_table['line'])


In [818]:

weekly_table[weekly_table['consensus_diff']>=10].sort_values('consensus_diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
31,T.Hill,MIA,6.0,87.6,104.2,1,101.1,114.5,97.1,87.6,75.5,12.1,True,115.151350,21.6,39.651350,100.0,24.5
22,M.Pittman,IND,6.0,51.8,53.3,1,54.9,46.7,47.3,51.8,68.5,16.7,False,49.707473,21.2,18.792527,49.6,18.9
40,A.Brown,PHI,6.0,56.1,53.1,1,47.9,60.2,52.0,56.1,68.5,12.4,False,54.711735,16.5,13.788265,54.3,14.2
54,C.Brate,TB,6.0,26.1,31.6,0,32.6,22.2,27.6,26.1,14.5,11.6,True,28.773953,13.1,14.273953,27.5,13.0
27,K.Allen,LAC,6.0,61.6,58.8,1,50.0,69.0,58.0,61.6,71.5,9.9,False,56.472997,13.5,15.027003,58.7,12.8
4,D.Robinson,BAL,6.0,49.9,50.6,1,45.2,52.3,47.9,49.9,36.5,13.4,True,49.733876,11.4,13.233876,49.2,12.7
17,D.Peoples-Jones,CLE,6.0,62.4,56.4,1,39.5,69.2,56.8,62.4,48.5,13.9,True,61.517241,8.3,13.017241,60.2,11.7
52,G.Kittle,SF,6.0,30.1,28.9,0,26.4,36.7,31.4,30.1,42.5,12.4,False,31.287793,11.1,11.212207,30.9,11.6
32,T.Sherfield,MIA,6.0,37.6,34.1,1,31.8,38.3,34.6,37.6,24.5,13.1,True,35.418009,10.1,10.918009,35.9,11.4
8,G.Davis,BUF,6.0,48.8,59.5,1,53.9,46.3,50.2,48.8,38.5,10.3,True,50.394237,11.7,11.894237,49.8,11.3


In [819]:
tnf = weekly_table[(weekly_table['team']=='SF')|(weekly_table['team']=='SEA')]

In [820]:
weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (35).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']


weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])


weekly_table['linear_diff']=abs(weekly_table['linear_projection']-weekly_table['line'])
weekly_table['beta_diff']=abs(weekly_table['beta_yards']-weekly_table['line'])
weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)
weekly_table['consensus_diff']=abs(weekly_table['consensus_proj']-weekly_table['line'])

In [821]:
weekly_table = tnf.append(weekly_table)

In [822]:
saturday = weekly_table[(weekly_table['team']=='IND')|(weekly_table['team']=='MIN')|(weekly_table['team']=='CLE')|(weekly_table['team']=='BAL')|(weekly_table['team']=='MIA')|(weekly_table['team']=='BUF')]

In [823]:
weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (36).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']


weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])


weekly_table['linear_diff']=abs(weekly_table['linear_projection']-weekly_table['line'])
weekly_table['beta_diff']=abs(weekly_table['beta_yards']-weekly_table['line'])
weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)
weekly_table['consensus_diff']=abs(weekly_table['consensus_proj']-weekly_table['line'])

In [824]:
weekly_table = tnf.append(saturday).append(weekly_table)

In [825]:
#agreement_df = 
weekly_table[(weekly_table['abs. diff']>=10)&(weekly_table['linear_diff']>=10)&(weekly_table['beta_diff']>=10)]

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
52,G.Kittle,SF,6.0,30.1,28.9,0,26.4,36.7,31.4,30.1,42.5,12.4,False,31.287793,11.1,11.212207,30.9,11.6
8,D.Robinson,BAL,6.0,49.9,50.6,1,45.2,52.3,47.9,49.9,35.5,14.4,True,49.733876,12.4,14.233876,49.2,13.7
12,G.Davis,BUF,6.0,48.8,59.5,1,53.9,46.3,50.2,48.8,37.5,11.3,True,50.394237,12.7,12.894237,49.8,12.3
46,M.Pittman,IND,6.0,51.8,53.3,1,54.9,46.7,47.3,51.8,68.5,16.7,False,49.707473,21.2,18.792527,49.6,18.9
71,T.Hill,MIA,6.0,87.6,104.2,1,101.1,114.5,97.1,87.6,75.5,12.1,True,115.151350,21.6,39.651350,100.0,24.5
72,T.Sherfield,MIA,6.0,37.6,34.1,1,31.8,38.3,34.6,37.6,24.5,13.1,True,35.418009,10.1,10.918009,35.9,11.4
2,M.Brown,ARI,6.0,72.0,86.3,1,88.5,75.7,74.2,72.0,48.0,24.0,True,74.365549,26.2,26.365549,73.5,25.5
33,J.Jeudy,DEN,6.0,48.3,45.7,1,50.3,58.5,47.6,48.3,60.5,12.2,False,47.270750,12.9,13.229250,47.7,12.8
46,P.Dorsett,HOU,6.0,21.9,18.7,1,18.3,22.0,21.0,21.9,34.5,12.6,False,20.983984,13.5,13.516016,21.3,13.2
52,J.McKinnon,KC,6.0,34.9,33.9,0,39.2,40.5,35.3,34.9,18.5,16.4,True,41.149415,16.8,22.649415,37.1,18.6


In [826]:
weekly_table[weekly_table['consensus_diff']>=10].sort_values('consensus_diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
2,M.Brown,ARI,6.0,72.0,86.3,1,88.5,75.7,74.2,72.0,48.0,24.0,True,74.365549,26.2,26.365549,73.5,25.5
71,T.Hill,MIA,6.0,87.6,104.2,1,101.1,114.5,97.1,87.6,75.5,12.1,True,115.151350,21.6,39.651350,100.0,24.5
64,J.Palmer,LAC,6.0,66.6,69.8,1,67.4,70.8,64.2,66.6,43.5,23.1,True,68.301628,20.7,24.801628,66.4,22.9
46,M.Pittman,IND,6.0,51.8,53.3,1,54.9,46.7,47.3,51.8,68.5,16.7,False,49.707473,21.2,18.792527,49.6,18.9
52,J.McKinnon,KC,6.0,34.9,33.9,0,39.2,40.5,35.3,34.9,18.5,16.4,True,41.149415,16.8,22.649415,37.1,18.6
78,D.Slayton,NYG,6.0,64.6,54.4,1,52.9,73.7,57.4,64.6,45.5,19.1,True,60.600190,11.9,15.100190,60.9,15.4
92,C.Brate,TB,6.0,26.1,31.6,0,32.6,22.2,27.6,26.1,13.5,12.6,True,28.773953,14.1,15.273953,27.5,14.0
8,D.Robinson,BAL,6.0,49.9,50.6,1,45.2,52.3,47.9,49.9,35.5,14.4,True,49.733876,12.4,14.233876,49.2,13.7
83,A.Brown,PHI,6.0,56.1,53.1,1,47.9,60.2,52.0,56.1,67.5,11.4,False,54.711735,15.5,12.788265,54.3,13.2
46,P.Dorsett,HOU,6.0,21.9,18.7,1,18.3,22.0,21.0,21.9,34.5,12.6,False,20.983984,13.5,13.516016,21.3,13.2


In [827]:
weekly_table = weekly_table[(weekly_table['team']!='NYG') & (weekly_table['team']!='WAS') & (weekly_table['team']!='LA') & (weekly_table['team']!='GB')]

In [828]:
w15_table = weekly_table.copy()

In [829]:
weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (37).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']


weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])


weekly_table['linear_diff']=abs(weekly_table['linear_projection']-weekly_table['line'])
weekly_table['beta_diff']=abs(weekly_table['beta_yards']-weekly_table['line'])
weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)
weekly_table['consensus_diff']=abs(weekly_table['consensus_proj']-weekly_table['line'])

In [830]:
snf15 = weekly_table.copy()


In [831]:
snf15

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
0,S.Barkley,NYG,6.0,18.3,21.2,0,23.3,13.5,19.3,18.3,21.5,3.2,False,18.237901,2.2,3.262099,18.6,2.9
1,D.Bellinger,NYG,6.0,23.7,21.2,0,21.1,27.7,24.3,23.7,22.5,1.2,True,21.437614,1.8,1.062386,23.1,0.6
2,D.Slayton,NYG,6.0,64.6,54.4,1,52.9,73.7,57.4,64.6,46.5,18.1,True,60.600190,10.9,14.100190,60.9,14.4
3,A.Gibson,WAS,6.0,21.9,19.6,0,17.3,26.0,22.9,21.9,19.5,2.4,True,21.850395,3.4,2.350395,22.2,2.7
4,C.Samuel,WAS,6.0,31.8,26.6,1,31.7,36.0,30.0,31.8,29.5,2.3,True,28.860919,0.5,0.639081,30.2,0.7
5,J.Dotson,WAS,6.0,26.2,30.4,1,25.7,22.3,27.0,26.2,29.5,3.3,False,27.523738,2.5,1.976262,26.9,2.6
6,L.Thomas,WAS,6.0,18.9,16.2,0,17.9,17.3,18.1,18.9,19.5,0.6,False,19.211736,1.4,0.288264,18.7,0.8
7,T.McLaurin,WAS,6.0,75.1,74.4,1,72.6,84.2,71.3,75.1,61.5,13.6,True,74.044658,9.8,12.544658,73.5,12.0


In [832]:
w15_table = w15_table.append(snf15)


w15_table

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
44,D.Metcalf,SEA,6.0,71.1,74.0,1,79.8,75.2,67.8,71.1,65.5,5.6,True,76.571970,2.3,11.071970,71.8,6.3
45,M.Goodwin,SEA,6.0,36.0,30.0,1,30.2,49.7,36.5,36.0,26.5,9.5,True,32.760143,10.0,6.260143,35.1,8.6
46,N.Fant,SEA,6.0,28.3,23.5,0,26.3,37.5,28.9,28.3,27.5,0.8,True,25.127478,1.4,2.372522,27.4,0.1
47,T.Lockett,SEA,6.0,63.7,57.9,1,58.2,71.3,58.3,63.7,64.5,0.8,False,65.489330,6.2,0.989330,62.5,2.0
48,W.Dissly,SEA,6.0,21.1,19.0,0,17.5,20.5,20.6,21.1,16.5,4.6,True,21.623788,4.1,5.123788,21.1,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,A.Gibson,WAS,6.0,21.9,19.6,0,17.3,26.0,22.9,21.9,19.5,2.4,True,21.850395,3.4,2.350395,22.2,2.7
4,C.Samuel,WAS,6.0,31.8,26.6,1,31.7,36.0,30.0,31.8,29.5,2.3,True,28.860919,0.5,0.639081,30.2,0.7
5,J.Dotson,WAS,6.0,26.2,30.4,1,25.7,22.3,27.0,26.2,29.5,3.3,False,27.523738,2.5,1.976262,26.9,2.6
6,L.Thomas,WAS,6.0,18.9,16.2,0,17.9,17.3,18.1,18.9,19.5,0.6,False,19.211736,1.4,0.288264,18.7,0.8


In [833]:
weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (39).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']


weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])


weekly_table['linear_diff']=abs(weekly_table['linear_projection']-weekly_table['line'])
weekly_table['beta_diff']=abs(weekly_table['beta_yards']-weekly_table['line'])
weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)
weekly_table['consensus_diff']=abs(weekly_table['consensus_proj']-weekly_table['line'])

In [834]:
mnf15 = weekly_table.copy()

In [835]:
mnf15

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
0,A.Dillon,GB,6.0,13.7,12.5,0,11.7,13.2,14.7,13.7,10.5,3.2,True,17.499569,4.2,6.999569,15.3,4.8
1,A.Jones,GB,6.0,22.7,22.9,0,24.4,25.3,24.3,22.7,21.5,1.2,True,26.421604,2.8,4.921604,24.5,3.0
2,C.Watson,GB,6.0,50.3,46.4,1,53.5,58.2,47.8,50.3,50.5,0.2,False,55.326700,2.7,4.826700,51.1,0.6
3,R.Tonyan,GB,6.0,22.9,24.5,0,24.6,20.2,23.3,22.9,22.5,0.4,True,22.970721,0.8,0.470721,23.1,0.6
4,A.Lazard,GB,6.0,50.9,58.9,1,55.6,55.8,53.4,50.9,43.5,7.4,True,49.025592,9.9,5.525592,51.1,7.6
5,B.Powell,LA,6.0,11.5,6.9,1,7.7,10.8,11.1,11.5,7.5,4.0,True,10.455039,3.6,2.955039,11.0,3.5
6,B.Skowronek,LA,6.0,27.9,32.3,1,29.5,23.7,28.5,27.9,31.5,3.6,False,31.415004,3.0,0.084996,29.3,2.2
7,T.Higbee,LA,6.0,22.5,22.8,0,18.9,23.8,23.7,22.5,25.5,3.0,False,21.568582,1.8,3.931418,22.6,2.9
8,T.Atwell,LA,6.0,33.9,30.8,1,23.2,39.5,33.3,33.9,36.5,2.6,False,32.447670,3.2,4.052330,33.2,3.3
9,V.Jefferson,LA,6.0,30.8,31.4,1,38.2,30.0,30.3,30.8,34.5,3.7,False,31.742999,4.2,2.757001,30.9,3.6


In [836]:
weekly_table = w15_table.append(mnf15)

In [837]:
weekly_table[weekly_table['player']=='A.Lazard']

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
4,A.Lazard,GB,6.0,50.9,58.9,1,55.6,55.8,53.4,50.9,43.5,7.4,True,49.025592,9.9,5.525592,51.1,7.6


In [838]:
w15_table[w15_table['player']=='M.Jones']

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
49,M.Jones,JAX,6.0,24.9,25.3,1,22.1,23.7,25.0,24.9,26.5,1.6,False,23.038895,1.5,3.461105,24.3,2.2


In [839]:
pp=w15_table.copy()

pp = pp[pp['player']=="A.Lazard"]


pp[['player','pYards','linear_projection','beta_yards','xYards/game','xYards_median','Yards/game','line']].sort_values('pYards',ascending=False).head(50).round(1)

,player,pYards,linear_projection,beta_yards,xYards/game,xYards_median,Yards/game,line


In [840]:
mnf15

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff
0,A.Dillon,GB,6.0,13.7,12.5,0,11.7,13.2,14.7,13.7,10.5,3.2,True,17.499569,4.2,6.999569,15.3,4.8
1,A.Jones,GB,6.0,22.7,22.9,0,24.4,25.3,24.3,22.7,21.5,1.2,True,26.421604,2.8,4.921604,24.5,3.0
2,C.Watson,GB,6.0,50.3,46.4,1,53.5,58.2,47.8,50.3,50.5,0.2,False,55.326700,2.7,4.826700,51.1,0.6
3,R.Tonyan,GB,6.0,22.9,24.5,0,24.6,20.2,23.3,22.9,22.5,0.4,True,22.970721,0.8,0.470721,23.1,0.6
4,A.Lazard,GB,6.0,50.9,58.9,1,55.6,55.8,53.4,50.9,43.5,7.4,True,49.025592,9.9,5.525592,51.1,7.6
5,B.Powell,LA,6.0,11.5,6.9,1,7.7,10.8,11.1,11.5,7.5,4.0,True,10.455039,3.6,2.955039,11.0,3.5
6,B.Skowronek,LA,6.0,27.9,32.3,1,29.5,23.7,28.5,27.9,31.5,3.6,False,31.415004,3.0,0.084996,29.3,2.2
7,T.Higbee,LA,6.0,22.5,22.8,0,18.9,23.8,23.7,22.5,25.5,3.0,False,21.568582,1.8,3.931418,22.6,2.9
8,T.Atwell,LA,6.0,33.9,30.8,1,23.2,39.5,33.3,33.9,36.5,2.6,False,32.447670,3.2,4.052330,33.2,3.3
9,V.Jefferson,LA,6.0,30.8,31.4,1,38.2,30.0,30.3,30.8,34.5,3.7,False,31.742999,4.2,2.757001,30.9,3.6


In [841]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==15]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

weekly_table = weekly_table.merge(results)

weekly_table['Over_Result'] = weekly_table['yards_gained']>weekly_table['line']

weekly_table['Win'] = (weekly_table['Over_Result'] == weekly_table['Over_Picked'])*1


weekly_table['week'] = 15


w15_table = weekly_table.copy()

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.78s/it]


In [842]:
w15_table.sort_values('abs. diff',ascending=False).head(25)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,...,Over_Picked,beta_yards,linear_diff,beta_diff,consensus_proj,consensus_diff,yards_gained,Over_Result,Win,week
39,M.Brown,ARI,6.0,72.0,86.3,1,88.5,75.7,74.2,72.0,...,True,74.365549,26.2,26.365549,73.5,25.5,19.0,False,0,15
91,J.Palmer,LAC,6.0,66.6,69.8,1,67.4,70.8,64.2,66.6,...,True,68.301628,20.7,24.801628,66.4,22.9,49.0,True,1,15
129,D.Slayton,NYG,6.0,64.6,54.4,1,52.9,73.7,57.4,64.6,...,True,60.600190,10.9,14.100190,60.9,14.4,23.0,False,0,15
26,M.Pittman,IND,6.0,51.8,53.3,1,54.9,46.7,47.3,51.8,...,False,49.707473,21.2,18.792527,49.6,18.9,60.0,False,1,15
83,J.McKinnon,KC,6.0,34.9,33.9,0,39.2,40.5,35.3,34.9,...,True,41.149415,16.8,22.649415,37.1,18.6,70.0,True,1,15
84,J.Smith-Schuster,KC,6.0,65.0,52.6,1,50.0,65.3,53.6,65.0,...,True,52.644663,3.1,2.144663,57.1,6.6,88.0,True,1,15
10,D.Robinson,BAL,6.0,49.9,50.6,1,45.2,52.3,47.9,49.9,...,True,49.733876,12.4,14.233876,49.2,13.7,29.0,False,0,15
20,D.Peoples-Jones,CLE,6.0,62.4,56.4,1,39.5,69.2,56.8,62.4,...,True,61.517241,8.3,13.017241,60.2,11.7,31.0,False,0,15
134,T.McLaurin,WAS,6.0,75.1,74.4,1,72.6,84.2,71.3,75.1,...,True,74.044658,9.8,12.544658,73.5,12.0,70.0,True,1,15
32,T.Sherfield,MIA,6.0,37.6,34.1,1,31.8,38.3,34.6,37.6,...,True,35.418009,10.1,10.918009,35.9,11.4,0.0,False,0,15


In [843]:
w15_table['Win'].mean()

0.5655172413793104

In [844]:
#player_disp['target_week'] = player_disp['week']+1

### Week 16

In [845]:
beta_table = pd.read_csv('wk16_beta.csv')


In [846]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==16]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w16_odds = pd.read_csv('betting_lines/tnf16_odds.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w16_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 




In [847]:

tnf16 = weekly_table.copy()

In [848]:
tnf16

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,C.Kirk,JAX,6.0,64.1,68.7,1,64.2,78.0,66.1,64.1,49.5,14.6,True
1,C.Uzomah,NYJ,6.0,15.9,12.0,0,12.6,18.7,16.4,15.9,9.5,6.4,True
2,E.Engram,JAX,6.0,38.6,41.9,0,36.4,46.7,41.5,38.6,39.5,0.9,False
3,G.Wilson,NYJ,6.0,65.9,67.9,1,59.4,89.5,69.9,65.9,53.5,12.4,True
4,J.Agnew,JAX,6.0,17.9,15.7,1,20.7,14.8,17.0,17.9,8.5,9.4,True
5,M.Jones,JAX,6.0,32.7,34.7,1,38.4,26.5,30.7,32.7,20.5,12.2,True
6,T.Conklin,NYJ,6.0,23.9,27.7,0,23.9,19.3,24.6,23.9,22.5,1.4,True
7,T.Etienne,JAX,6.0,13.6,10.1,0,13.1,13.5,13.6,13.6,13.5,0.1,True
8,Z.Jones,JAX,6.0,65.5,65.9,1,64.8,75.8,64.0,65.5,44.5,21.0,True


In [849]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==16]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w16_odds = pd.read_csv('betting_lines/wk16_sat.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w16_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


wk16_saturday = weekly_table.copy()

wk16_saturday

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,A.Thielen,MIN,6.0,46.9,47.7,1,52.6,44.7,43.7,46.9,46.5,0.4,True
1,B.Aiyuk,SF,6.0,48.6,48.8,1,49.8,48.5,45.6,48.6,50.5,1.9,False
2,C.McCaffrey,SF,6.0,36.9,41.8,0,43.7,44.5,40.7,36.9,36.5,0.4,True
3,C.Patterson,ATL,6.0,13.0,13.3,0,8.9,6.2,12.6,13.0,7.5,5.5,True
4,C.Samuel,WAS,6.0,32.2,28.1,1,33.2,35.0,30.4,32.2,30.5,1.7,True
5,D.Adams,LV,6.0,77.8,94.3,1,95.0,102.8,88.0,77.8,71.5,6.3,True
6,D.Bellinger,NYG,6.0,21.9,21.8,0,21.7,21.7,22.4,21.9,27.5,5.6,False
7,D.Cook,MIN,6.0,21.6,16.8,0,19.0,24.3,20.9,21.6,17.5,4.1,True
8,D.Knox,BUF,6.0,39.5,41.8,0,49.8,47.2,41.6,39.5,29.5,10.0,True
9,D.London,ATL,6.0,50.5,55.3,1,43.7,42.8,46.9,50.5,45.5,5.0,True


In [850]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==16]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w16_odds = pd.read_csv('betting_lines/wk16_sun.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w16_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


wk16_sunday = weekly_table.copy()

wk16_sunday

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,A.Dillon,GB,6.0,15.5,14.4,0,17.1,17.5,17.3,15.5,11.5,4.0,True
1,A.Jones,GB,6.0,24.0,22.4,0,22.9,29.0,25.4,24.0,28.5,4.5,False
2,A.Lazard,GB,6.0,50.2,54.0,1,44.8,47.8,48.0,50.2,43.5,6.7,True
3,C.Godwin,TB,6.0,69.5,71.8,1,67.3,69.5,64.7,69.5,67.5,2.0,True
4,C.Watson,GB,6.0,56.4,50.7,1,53.5,63.8,52.1,56.4,53.5,2.9,True
5,D.Hopkins,ARI,6.0,69.8,72.6,1,74.9,75.2,67.1,69.8,58.5,11.3,True
6,G.Dulcich,DEN,6.0,34.9,39.4,0,39.3,31.7,34.9,34.9,31.5,3.4,True
7,J.Conner,ARI,6.0,22.3,24.2,0,26.1,20.8,23.4,22.3,19.5,2.8,True
8,J.Jeudy,DEN,6.0,51.2,43.7,1,50.3,62.2,47.9,51.2,57.5,6.3,False
9,J.Waddle,MIA,6.0,65.0,66.4,1,59.1,65.0,60.4,65.0,64.5,0.5,True


In [851]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==16]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w16_odds = pd.read_csv('betting_lines/wk16_mnf.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w16_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


wk16_mnf = weekly_table.copy()

In [852]:
weekly_table = tnf16.append(wk16_saturday).append(wk16_sunday).append(wk16_mnf).sort_values('abs. diff',ascending=False)

weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])

weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)

weekly_table['consensus_diff'] = abs(weekly_table['consensus_proj']-weekly_table['line'])

In [853]:
weekly_table.sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff
0,Z.Jones,JAX,6.0,65.5,65.9,1,64.8,75.8,64.0,65.5,44.5,21.0,True,64.614987,64.7,20.2
1,J.Palmer,LAC,6.0,58.0,61.0,1,61.2,61.3,56.4,58.0,37.5,20.5,True,57.940509,57.4,19.9
2,D.Carter,LAC,6.0,30.3,29.1,1,22.6,33.2,30.2,30.3,10.5,19.8,True,29.213452,29.9,19.4
3,T.Hill,MIA,6.0,72.3,90.5,1,82.9,94.7,83.1,72.3,90.5,18.2,False,80.256517,78.6,11.9
4,A.Pierce,IND,6.0,22.7,23.1,1,5.4,22.8,23.5,22.7,37.5,14.8,False,22.940006,23.0,14.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,A.Thielen,MIN,6.0,46.9,47.7,1,52.6,44.7,43.7,46.9,46.5,0.4,True,49.269107,46.6,0.1
75,T.Higbee,LA,6.0,27.1,26.6,0,22.6,28.3,27.3,27.1,27.5,0.4,False,24.958949,26.5,1.0
76,A.Ekeler,LAC,6.0,35.2,37.8,0,36.7,40.0,37.0,35.2,35.5,0.3,False,38.019110,36.7,1.2
77,T.Etienne,JAX,6.0,13.6,10.1,0,13.1,13.5,13.6,13.6,13.5,0.1,True,13.873353,13.7,0.2


In [854]:
weekly_table[weekly_table['player']=='G.Wilson']

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff
9,G.Wilson,NYJ,6.0,65.9,67.9,1,59.4,89.5,69.9,65.9,53.5,12.4,True,69.921809,68.6,15.1


In [855]:
player__ = 'D.Knox'


player_analyzer2 = player_analyzer.copy()

player_by_week = player_analyzer2[player_analyzer2['receiver_player_name']==player__]

#team = 'MIA'

#player_by_week = player_by_week[player_by_week['team']==team]


player_disp = player_by_week[['full_name','team','week','offense_snaps','xYards','yards_gained','targets','target_share','aDOT','complete_pass','cp','yards_after_catch','xyac_mean_yardage']].round(2).drop_duplicates()

player_mean = player_disp.tail(6)['xYards'].mean()

player_sd = player_disp.tail(6)['xYards'].std()

#player_disp

In [856]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==16]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

weekly_table = weekly_table.merge(results)

weekly_table['Over_Result'] = weekly_table['yards_gained']>weekly_table['line']

weekly_table['Win'] = (weekly_table['Over_Result'] == weekly_table['Over_Picked'])*1


weekly_table['week'] = 16


w16_table = weekly_table.copy()

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.98s/it]


### Week 17

In [857]:
beta_table = pd.read_csv('wk17_beta.csv')


In [858]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==17]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w16_odds = pd.read_csv('betting_lines/tnf_wk17.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w16_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 

tnf17 = weekly_table.copy()




In [859]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==17]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
weekly_odds = pd.read_csv('betting_lines/wk17_sun.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = weekly_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 

wk17_sunday = weekly_table.copy() 




In [860]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==17]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
weekly_odds = pd.read_csv('betting_lines/wk17_mon.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = weekly_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 

mnf17 = weekly_table.copy()




In [861]:
mnf17

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,D.Knox,BUF,6.0,38.8,41.3,0,48.3,44.0,40.2,38.8,37.5,1.3,True
1,D.Singletary,BUF,6.0,13.5,12.3,0,12.9,11.7,14.1,13.5,10.5,3.0,True
2,G.Davis,BUF,6.0,49.3,54.0,1,54.3,42.2,46.0,49.3,45.5,3.8,True
3,H.Hurst,CIN,6.0,38.0,41.4,0,33.7,37.0,37.8,38.0,29.5,8.5,True
4,I.McKenzie,BUF,6.0,33.9,36.0,1,33.6,30.8,32.9,33.9,23.5,10.4,True
5,J.Chase,CIN,6.0,76.0,88.4,1,84.2,102.8,84.9,76.0,82.5,6.5,False
6,J.Cook,BUF,6.0,17.6,16.8,0,13.0,13.0,16.9,17.6,10.5,7.1,True
7,J.Mixon,CIN,6.0,29.6,30.6,0,26.7,36.3,32.1,29.6,25.5,4.1,True
8,S.Diggs,BUF,6.0,53.9,63.5,1,58.3,56.7,55.9,53.9,72.5,18.6,False
9,T.Boyd,CIN,6.0,30.1,30.6,1,29.6,29.0,29.5,30.1,35.5,5.4,False


In [862]:
weekly_table = mnf17.append(tnf17).append(wk17_sunday)

In [863]:
#weekly_table = tnf16.append(wk16_saturday).append(wk16_sunday).append(wk16_mnf).sort_values('abs. diff',ascending=False)

weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])

weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)

weekly_table['consensus_diff'] = abs(weekly_table['consensus_proj']-weekly_table['line'])

weekly_table['beta_yards'] = weekly_table['beta_yards'].round(1)

In [864]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==17]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

weekly_table = weekly_table.merge(results)

weekly_table['Over_Result'] = weekly_table['yards_gained']>weekly_table['line']

weekly_table['Win'] = (weekly_table['Over_Result'] == weekly_table['Over_Picked'])*1


weekly_table['week'] = 17


w17_table = weekly_table.copy()

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.92s/it]


### Week 18

In [865]:
beta_table = pd.read_csv('wk18_beta.csv')


In [866]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==18]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w18_odds = pd.read_csv('betting_lines/sat_18.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w18_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 

sat_18 = weekly_table.copy()




In [867]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==18]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w18_odds = pd.read_csv('betting_lines/sun_18.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w18_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 

sun_18 = weekly_table.copy()




In [868]:
weekly_table = sat_18.append(sun_18)

In [869]:
#weekly_table = tnf16.append(wk16_saturday).append(wk16_sunday).append(wk16_mnf).sort_values('abs. diff',ascending=False)

weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])

weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)

weekly_table['consensus_diff'] = abs(weekly_table['consensus_proj']-weekly_table['line'])

weekly_table['beta_yards'] = weekly_table['beta_yards'].round(1)

weekly_table.sort_values('consensus_diff',ascending=False).head(20)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff
54,J.Waddle,MIA,6.0,65.8,62.1,1,55.7,72.3,60.8,65.8,47.5,18.3,True,68.7,65.1,17.6
63,S.Diggs,BUF,6.0,53.9,63.5,1,58.3,56.7,55.9,53.9,71.5,17.6,False,53.5,54.4,17.1
46,G.Wilson,NYJ,6.0,71.8,79.5,1,72.0,80.2,72.4,71.8,55.5,16.3,True,72.7,72.3,16.8
13,K.Toney,KC,6.0,24.7,17.0,1,11.2,25.5,21.4,24.7,34.5,9.8,False,13.9,20.0,14.5
70,T.Pollard,DAL,6.0,30.0,29.7,0,31.2,39.5,32.8,30.0,16.5,13.5,True,30.0,30.9,14.4
12,J.Smith-Schuster,KC,6.0,42.6,42.0,1,29.1,47.2,41.7,42.6,54.5,11.9,False,42.4,42.2,12.3
22,T.Kelce,KC,6.0,63.3,66.5,0,74.1,74.2,63.7,63.3,77.5,14.2,False,69.7,65.6,11.9
1,C.Kirk,JAX,6.0,51.8,54.3,1,53.9,55.0,50.7,51.8,62.5,10.7,False,50.2,50.9,11.6
68,T.Hill,MIA,6.0,76.0,93.4,1,82.3,89.8,82.9,76.0,66.5,9.5,True,74.8,77.9,11.4
43,E.Moore,NYJ,6.0,39.5,38.8,1,30.4,35.8,36.1,39.5,27.5,12.0,True,37.2,37.6,10.1


In [870]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==18]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

weekly_table = weekly_table.merge(results)

weekly_table['Over_Result'] = weekly_table['yards_gained']>weekly_table['line']

weekly_table['Win'] = (weekly_table['Over_Result'] == weekly_table['Over_Picked'])*1


weekly_table['week'] = 18


w18_table = weekly_table.copy()

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.94s/it]


### Wild Card Round

In [871]:
beta_table = pd.read_csv('WC_beta.csv')


In [872]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==19]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
w18_odds = pd.read_csv('betting_lines/wc_lines.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w18_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 





In [873]:
#weekly_table = tnf16.append(wk16_saturday).append(wk16_sunday).append(wk16_mnf).sort_values('abs. diff',ascending=False)

weekly_table = weekly_table.merge(beta_table[['player','team','beta_yards']], on = ['player','team'])

weekly_table['consensus_proj']=((weekly_table['beta_yards']+weekly_table['pYards']+weekly_table['linear_projection'])/3).round(1)

weekly_table['consensus_diff'] = abs(weekly_table['consensus_proj']-weekly_table['line'])

weekly_table['beta_yards'] = weekly_table['beta_yards'].round(1)

weekly_table['beta_diff'] = abs(weekly_table['beta_yards']-weekly_table['line'])


weekly_table.sort_values('consensus_diff',ascending=False).head(20)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff,beta_diff
34,S.Diggs,BUF,6.0,67.9,71.9,1,82.2,66.0,63.5,67.9,80.0,12.1,False,53.5,61.6,18.4,26.5
21,J.Jefferson,MIN,6.0,76.6,93.2,1,99.4,97.8,85.6,76.6,95.5,18.9,False,77.3,79.8,15.7,18.2
15,E.Engram,JAX,6.0,56.5,49.0,0,45.1,68.3,52.8,56.5,41.5,15.0,True,57.8,55.7,14.2,16.3
2,B.Aiyuk,SF,6.0,54.8,59.3,1,58.1,60.5,55.2,54.8,45.5,9.3,True,62.7,57.6,12.1,17.2
30,N.Brown,DAL,6.0,36.8,40.0,1,42.2,28.5,34.1,36.8,21.0,15.8,True,27.5,32.8,11.8,6.5
24,K.Osborn,MIN,6.0,59.3,48.4,1,36.0,67.5,52.2,59.3,40.5,18.8,True,42.0,51.2,10.7,1.5
19,I.McKenzie,BUF,6.0,37.8,39.8,1,33.6,34.0,35.9,37.8,23.5,14.3,True,29.0,34.2,10.7,5.5
13,D.Slayton,NYG,6.0,43.4,43.6,1,43.5,51.8,44.2,43.4,51.5,8.1,False,41.2,42.9,8.6,10.3
12,D.Schultz,DAL,6.0,45.9,56.8,0,56.5,44.5,48.3,45.9,38.5,7.4,True,47.2,47.1,8.6,8.7
32,R.White,TB,6.0,20.4,21.4,0,22.7,18.3,21.1,20.4,14.5,5.9,True,27.1,22.9,8.4,12.6


In [874]:
weekly_table.sort_values('consensus_diff',ascending=False).head(50)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff,beta_diff
34,S.Diggs,BUF,6.0,67.9,71.9,1,82.2,66.0,63.5,67.9,80.0,12.1,False,53.5,61.6,18.4,26.5
21,J.Jefferson,MIN,6.0,76.6,93.2,1,99.4,97.8,85.6,76.6,95.5,18.9,False,77.3,79.8,15.7,18.2
15,E.Engram,JAX,6.0,56.5,49.0,0,45.1,68.3,52.8,56.5,41.5,15.0,True,57.8,55.7,14.2,16.3
2,B.Aiyuk,SF,6.0,54.8,59.3,1,58.1,60.5,55.2,54.8,45.5,9.3,True,62.7,57.6,12.1,17.2
30,N.Brown,DAL,6.0,36.8,40.0,1,42.2,28.5,34.1,36.8,21.0,15.8,True,27.5,32.8,11.8,6.5
24,K.Osborn,MIN,6.0,59.3,48.4,1,36.0,67.5,52.2,59.3,40.5,18.8,True,42.0,51.2,10.7,1.5
19,I.McKenzie,BUF,6.0,37.8,39.8,1,33.6,34.0,35.9,37.8,23.5,14.3,True,29.0,34.2,10.7,5.5
13,D.Slayton,NYG,6.0,43.4,43.6,1,43.5,51.8,44.2,43.4,51.5,8.1,False,41.2,42.9,8.6,10.3
12,D.Schultz,DAL,6.0,45.9,56.8,0,56.5,44.5,48.3,45.9,38.5,7.4,True,47.2,47.1,8.6,8.7
32,R.White,TB,6.0,20.4,21.4,0,22.7,18.3,21.1,20.4,14.5,5.9,True,27.1,22.9,8.4,12.6


In [875]:
weekly_table.corr()[['line']].sort_values('line',ascending=False)

,line
line,1.000000
linear_projection,0.956434
consensus_proj,0.956116
xYards/game,0.947073
pYards,0.942824
gb_projection,0.942824
xYards_median,0.936977
Yards/game,0.928089
beta_yards,0.928061
WR,0.665660


In [876]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    #data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==19]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

weekly_table = weekly_table.merge(results)

weekly_table['Over_Result'] = weekly_table['yards_gained']>weekly_table['line']

weekly_table['Win'] = (weekly_table['Over_Result'] == weekly_table['Over_Picked'])*1


weekly_table['week'] = 19








wc_table = weekly_table.copy()

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.85s/it]


In [877]:
wc_table

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,...,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff,beta_diff,yards_gained,Over_Result,Win,week
0,A.Ekeler,LAC,6.0,27.4,25.3,0,20.8,35.2,29.0,27.4,...,9.1,False,36.1,30.8,5.7,0.4,8.0,False,1,19
1,A.Thielen,MIN,6.0,36.4,40.1,1,40.5,27.2,33.7,36.4,...,1.1,False,45.3,38.5,1.0,7.8,50.0,True,0,19
2,B.Aiyuk,SF,6.0,54.8,59.3,1,58.1,60.5,55.2,54.8,...,9.3,True,62.7,57.6,12.1,17.2,73.0,True,1,19
3,C.Godwin,TB,6.0,65.9,64.6,1,65.5,73.0,62.3,65.9,...,2.6,False,73.7,67.3,1.2,5.2,85.0,True,0,19
4,C.Kirk,JAX,6.0,59.7,55.2,1,53.9,63.8,54.3,59.7,...,1.2,True,50.2,54.7,3.8,8.3,78.0,True,1,19
5,C.Lamb,DAL,6.0,73.9,72.3,1,63.2,83.7,70.0,73.9,...,0.6,False,75.2,73.0,1.5,0.7,68.0,False,1,19
6,C.Otton,TB,6.0,24.9,30.5,0,31.6,18.3,25.7,24.9,...,1.4,True,25.2,25.3,1.8,1.7,58.0,True,1,19
7,C.Parkinson,SEA,6.0,23.5,20.8,0,18.4,25.0,23.2,23.5,...,5.0,True,21.0,22.6,4.1,2.5,14.0,False,0,19
8,D.Cook,MIN,6.0,18.5,15.4,0,16.0,22.5,19.5,18.5,...,2.0,True,22.4,20.1,3.6,5.9,10.0,False,0,19
9,D.Knox,BUF,6.0,32.8,33.2,0,29.4,34.5,32.8,32.8,...,1.2,False,42.0,35.9,1.9,8.0,20.0,False,1,19


### Divisonal Round

In [885]:
beta_table = pd.read_csv('DR_beta.csv')


In [910]:
beta_table_prev

,Unnamed: 0,receiver_player_name,team,beta_yards
0,0,A.Green,ARI,15.758783
1,1,E.Benjamin,ARI,14.338279
2,2,G.Dortch,ARI,35.346982
3,3,J.Conner,ARI,26.141655
4,4,M.Brown,ARI,53.640245
...,...,...,...,...
404,404,D.Brown,WAS,4.672596
405,405,C.Sims,WAS,8.257868
406,406,C.Turner,WAS,4.548942
407,407,B.Robinson,WAS,7.944407


In [894]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==20]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

In [926]:
preview = trailing_weeks.sort_values('pYards',ascending=False)[['receiver_player_name','posteam','pYards']]

beta_table_prev = beta_table.rename(columns={'player':'receiver_player_name','team':'posteam'})

prev_table = preview.merge(beta_table_prev, on=['receiver_player_name','posteam'])

In [927]:
prev_table = prev_table[['receiver_player_name','posteam','pYards','beta_yards']].round(1)

In [937]:
prev_table[(prev_table['posteam']=='BUF')|(prev_table['posteam']=='CIN')]

,receiver_player_name,posteam,pYards,beta_yards
10,J.Chase,CIN,72.2,79.1
14,S.Diggs,BUF,66.1,53.5
33,T.Higgins,CIN,52.5,56.1
34,G.Davis,BUF,51.7,48.0
66,I.McKenzie,BUF,37.8,29.0
81,D.Knox,BUF,34.7,42.0
94,H.Hurst,CIN,32.5,36.5
103,T.Boyd,CIN,30.8,27.6
121,J.Mixon,CIN,26.4,31.9
133,K.Shakir,BUF,23.4,5.0





weekly_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
weekly_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
weekly_table.set_index(['player','team'],inplace=True)
 
weekly_odds = pd.read_csv('betting_lines/dr_lines.csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = weekly_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,right_index=True,left_index=True)
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['gb_projection'] - weekly_table['line'])



weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['gb_projection']>weekly_table ['line']
 





In [939]:
player__ = 'T.Higgins'


player_analyzer2 = player_analyzer.copy()

player_by_week = player_analyzer2[player_analyzer2['receiver_player_name']==player__]

#team = 'MIA'

#player_by_week = player_by_week[player_by_week['team']==team]


player_disp = player_by_week[['full_name','team','week','offense_snaps','xYards','yards_gained','targets','target_share','aDOT','complete_pass','cp','yards_after_catch','xyac_mean_yardage']].round(2).drop_duplicates()

player_mean = player_disp.tail(6)['xYards'].mean()

player_sd = player_disp.tail(6)['xYards'].std()

player_disp

,full_name,team,week,offense_snaps,xYards,yards_gained,targets,target_share,aDOT,complete_pass,cp,yards_after_catch,xyac_mean_yardage
0,Tee Higgins,CIN,1.0,26.0,14.96,27.0,2.0,0.04,9.00,2.0,1.18,9.0,8.47
1,Tee Higgins,CIN,2.0,62.0,82.33,71.0,10.0,0.29,9.60,6.0,6.11,18.0,23.47
2,Tee Higgins,CIN,3.0,46.0,62.84,93.0,7.0,0.20,14.29,5.0,3.61,13.0,28.57
3,Tee Higgins,CIN,4.0,49.0,84.69,124.0,9.0,0.28,10.67,7.0,5.88,72.0,43.91
4,Tee Higgins,CIN,5.0,10.0,0.00,0.0,0.0,NaN,NaN,0.0,0.00,0.0,0.00
5,Tee Higgins,CIN,6.0,49.0,62.54,47.0,9.0,0.25,6.78,6.0,6.78,12.0,22.38
6,Tee Higgins,CIN,7.0,57.0,65.25,93.0,7.0,0.17,10.86,5.0,4.58,35.0,17.08
7,Tee Higgins,CIN,8.0,52.0,45.82,49.0,6.0,0.18,14.50,3.0,3.44,11.0,14.53
8,Tee Higgins,CIN,9.0,46.0,56.45,60.0,8.0,0.27,6.88,7.0,5.72,6.0,25.38
10,Tee Higgins,CIN,11.0,52.0,110.29,148.0,13.0,0.35,11.15,9.0,7.96,44.0,37.68


In [940]:
player_mean

57.32500000000001

## Overall Results

In [919]:
szn_results = w8_results.append(w9_table).append(w10_table).append(w11_table).append(w12_table).append(w13_table).append(w14_table).append(w15_table).append(w16_table).append(w17_table).append(w18_table).append(wc_table)

szn_results['abs_diff'] = abs(szn_results['pYards']-szn_results['line'])

szn_results['smash'] = szn_results['abs_diff']>=10

smash_df = szn_results[szn_results['smash']==True]


In [920]:
print('R Values:\n')


print('Yards and projection: {}'.format(round(szn_results['pYards'].corr(szn_results['yards_gained']),3)))
print('Yards and betting line: {}'.format(round(szn_results['line'].corr(szn_results['yards_gained']),3)))
print('Projection and betting line: {}'.format(round(szn_results['pYards'].corr(szn_results['line']),3)))


R Values:

Yards and projection: 0.527
Yards and betting line: 0.584
Projection and betting line: 0.94


In [921]:


print('Overall Record: {}-{} ({}%)'.format(szn_results['Win'].sum(),szn_results.shape[0]-szn_results['Win'].sum(),round(szn_results['Win'].sum()/szn_results.shape[0]*100,1)))

print('Plays Record (10+ yard difference): {}-{} ({}%)'.format(smash_df['Win'].sum(),smash_df.shape[0]-smash_df['Win'].sum(),round(smash_df['Win'].sum()/smash_df.shape[0]*100,1)))

print('\n-----\n')
 
mae = mean_absolute_error(szn_results['pYards'], szn_results['yards_gained'])
    
print("Mean Absolute Error - Projections:")
print('{}\n'.format(round(mae,1)))


mae = mean_absolute_error(szn_results['yards_gained'],szn_results['line'])
 
print("Mean Absolute Error - Betting line:")
print(round(mae,1))

Overall Record: 538-527 (50.5%)
Plays Record (10+ yard difference): 84-90 (48.3%)

-----

Mean Absolute Error - Projections:
22.0

Mean Absolute Error - Betting line:
21.1


In [922]:
MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:")
print('{}\n'.format(round(RMSE,1)))


MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['line'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting line:")
print(round(RMSE,1))

Root Mean Square Error - Projections:
30.0

Root Mean Square Error - Betting line:
28.8


In [923]:
print('WEEKLY RESULTS')

print('\nOverall')
for week in szn_results['week'].unique():
    w = szn_results[szn_results['week']==week]['Win'].sum()
    p = szn_results[szn_results['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))


print('\nPlays')

for week in smash_df['week'].unique():
    w = smash_df[smash_df['week']==week]['Win'].sum()
    p = smash_df[smash_df['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))

WEEKLY RESULTS

Overall
Week 8: 53-47 (53.0%)
Week 9: 39-47 (45.3%)
Week 10: 43-51 (45.7%)
Week 11: 44-35 (55.7%)
Week 12: 53-52 (50.5%)
Week 13: 62-59 (51.2%)
Week 14: 48-54 (47.1%)
Week 15: 82-63 (56.6%)
Week 16: 43-33 (56.6%)
Week 17: 22-24 (47.8%)
Week 18: 28-43 (39.4%)
Week 19: 21-19 (52.5%)

Plays
Week 8: 7-8 (46.7%)
Week 9: 8-7 (53.3%)
Week 10: 3-8 (27.3%)
Week 11: 11-6 (64.7%)
Week 12: 9-12 (42.9%)
Week 13: 12-8 (60.0%)
Week 14: 11-8 (57.9%)
Week 15: 11-9 (55.0%)
Week 16: 5-12 (29.4%)
Week 17: 2-4 (33.3%)
Week 18: 3-5 (37.5%)
Week 19: 2-3 (40.0%)


In [924]:
wc_table.sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,...,abs. diff,Over_Picked,beta_yards,consensus_proj,consensus_diff,beta_diff,yards_gained,Over_Result,Win,week
20,J.Jefferson,MIN,6.0,76.6,93.2,1,99.4,97.8,85.6,76.6,...,18.9,False,77.3,79.8,15.7,18.2,47.0,False,1,19
23,K.Osborn,MIN,6.0,59.3,48.4,1,36.0,67.5,52.2,59.3,...,18.8,True,42.0,51.2,10.7,1.5,20.0,False,0,19
29,N.Brown,DAL,6.0,36.8,40.0,1,42.2,28.5,34.1,36.8,...,15.8,True,27.5,32.8,11.8,6.5,18.0,False,0,19
15,E.Engram,JAX,6.0,56.5,49.0,0,45.1,68.3,52.8,56.5,...,15.0,True,57.8,55.7,14.2,16.3,93.0,True,1,19
33,S.Diggs,BUF,6.0,67.9,71.9,1,82.2,66.0,63.5,67.9,...,12.1,False,53.5,61.6,18.4,26.5,114.0,True,0,19
2,B.Aiyuk,SF,6.0,54.8,59.3,1,58.1,60.5,55.2,54.8,...,9.3,True,62.7,57.6,12.1,17.2,73.0,True,1,19
0,A.Ekeler,LAC,6.0,27.4,25.3,0,20.8,35.2,29.0,27.4,...,9.1,False,36.1,30.8,5.7,0.4,8.0,False,1,19
28,M.Jones,JAX,6.0,36.3,42.4,1,51.3,26.8,34.7,36.3,...,8.8,True,32.7,34.6,7.1,5.2,29.0,True,1,19
13,D.Slayton,NYG,6.0,43.4,43.6,1,43.5,51.8,44.2,43.4,...,8.1,False,41.2,42.9,8.6,10.3,88.0,True,0,19
12,D.Schultz,DAL,6.0,45.9,56.8,0,56.5,44.5,48.3,45.9,...,7.4,True,47.2,47.1,8.6,8.7,95.0,True,1,19


In [930]:
szn_results['Over_Picked'].value_counts()

True     643
False    422
Name: Over_Picked, dtype: int64

In [931]:
szn_results['Over_Result'].value_counts()

True     538
False    527
Name: Over_Result, dtype: int64

In [932]:
over_w = szn_results[szn_results['Over_Picked']==1]['Win'].mean()
under_w = szn_results[szn_results['Over_Picked']==0]['Win'].mean()

print('Overs: {}%'.format(round(over_w*100,1)))
print('Under: {}%'.format(round(under_w*100,1)))

Overs: 50.9%
Under: 50.0%


In [933]:
over_w = smash_df[smash_df['Over_Picked']==1]['Win'].mean()
under_w = smash_df[smash_df['Over_Picked']==0]['Win'].mean()

print('Smash Overs: {}%'.format(round(over_w*100,1)))
print('Smash Under: {}%'.format(round(under_w*100,1)))

Smash Overs: 45.0%
Smash Under: 52.7%


In [ ]:
szn_results['the_blend'] = (szn_results['xYards_median']+szn_results['pYards'])/2

szn_results.corr().sort_values('yards_gained',ascending=False)

In [ ]:
szn_results[szn_results['gb_projection'].isna()==True]